In [1]:
# RUN Main import block and TODO list

# TODO: see how uri calculated the ridges

# TODO: Perform Histogram equalization - start with it
# TODO: 
# take integral from the Highest peak+-0.005 divide by integral of the entire graph 
# This will be the peakness measure for the PSD ==> The desired ridge index
# TODO:
# take integral from the Highest peak+-0.005 divide by integral of the entire graph - it's the peakness measure for the PSD
# must select a peak above a min threshold in order to ignore noisy frequency
# must ignore peaks above a certain threshold in order to detect meaningful frequency
# run the PSD in moving windows every 200 px (deduced from the below PSD pointing to a freq of 1/0.02=50-> times 4= 200px)
# and medianf the result of the windows
# TODO:
# Another alternative: (with Yariv)
# Run PSD column by column - get the phase, freq, peakness and reconstruct an artificial ridge slice
# from this - reconstruct a "clean" artificial ridge image

%matplotlib inline
import matplotlib.image as img
import matplotlib.pyplot as plt
import numpy as np
import urllib.request
import os
import shutil
import glob
from scipy import ndimage
from scipy import signal
#import cv2

from PIL import Image, ImageDraw, ImageFont

import mahotas as mh
from mahotas import polygon
# import pymorph as pm

import networkx as nx

from scipy import ndimage as nd
import skimage.transform as transform
import skimage.morphology as mp
import skimage.io as sio
import scipy.misc as sm
from skimage.filters import threshold_otsu, threshold_adaptive
from skimage.feature import hessian_matrix, hessian_matrix_eigvals
from skimage import exposure
from skimage import data, img_as_float
from sklearn.metrics import precision_recall_fscore_support

from bisect import bisect_left
import math
import warnings
import csv
import tensorflow as tf
from time import gmtime, strftime

# Imports the Google Cloud client library
from google.cloud import storage


In [2]:
# RUN GCP and INIT

files_in_gcp=True
bucket_name = "papyrus-thesis"
all_root = "thesis-papyri/"
imgs_root = all_root + "PAPYRI/" # "/Volumes/250GB/PAPYRI/"
cropped_root = all_root + "cropped/" # cropped=front cropped2=back # "/Volumes/250gb/cropped2"
matched_root = all_root + "non593_front/matched/" # "/thesis-papyri/matched593/" # "/Volumes/250gb/cropped2"
local_temp_root = "temp/"
match_path = all_root + "non593_front/matched/" # "thesis-papyri/matched593_20190105/"
ux_path = all_root + "non593_front/ux/" # "thesis-papyri/ux593_20190105/"

#BASIC CROP FRAME
X_START = 1000
X_END = 6000
Y_START = 800
Y_END = 4300
BG_2_OBJ_RATIO = 0.91
CUBE_SIZE = 250
EDGE_GAP = 50
ROOT_FOLDER = "/media/1KGB_ILAN/papyrus/"
#ROOT_FOLDER = "/Users/il239838/Downloads/private/Thesis/Papyrus/PX303/files/"
VOL_FOLDER = "/media/1KGB_ILAN/papyrus/files/"
#VOL_FOLDER = "/Volumes/250GB/"
LEARNING_RATE = 0.001
BATCHES = 800
BATCH_SIZE = 50
BREAK_VAL = 1000

# Instantiates a client
storage_client = storage.Client()

def list_blobs_with_prefix(prefix):
    bucket = storage_client.get_bucket(bucket_name)
    blobs = bucket.list_blobs(prefix=prefix, delimiter='/')

    for blob in blobs:
        if blob.name[-1] != '/':
            yield blob.name
        
    #     if delimiter:
    #         print('Prefixes:')
    #         for prefix in blobs.prefixes:
    #             print(prefix)

def download_blob(source_blob_name, destination_file_name):
    bucket = storage_client.get_bucket(bucket_name)
    blob = bucket.blob(source_blob_name)

    blob.download_to_filename(destination_file_name)

    print('Blob {} downloaded to {}.'.format(
        source_blob_name,
        destination_file_name))

def upload_blob(source_file_name, destination_blob_name):
    bucket = storage_client.get_bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)

    blob.upload_from_filename(source_file_name)

    print('File {} uploaded to {}.'.format(
        source_file_name,
        destination_blob_name))
    
def gcp_img_write(img_path, target_path):
    # import pdb; pdb.set_trace()
    fname = img_path[img_path.rfind("/") + 1:]
    destination = target_path + img_path[img_path.rfind("/") + 1:]
    result = upload_blob(img_path, destination)
    os.remove(img_path)
    return destination

def gcp_img_read(img_path):
    # import pdb; pdb.set_trace()
    fname = img_path[img_path.rfind("/") + 1:]
    download_blob(img_path, local_temp_root + fname)
    result = img.imread(local_temp_root + fname)
    os.remove(local_temp_root + fname)
    return result

def gcp_np_load(img_path):
    # import pdb; pdb.set_trace()
    fname = img_path[img_path.rfind("/") + 1:]
    download_blob(img_path, local_temp_root + fname)
    result = np.load(local_temp_root + fname)
    os.remove(local_temp_root + fname)
    return result
    
# Simple crop by x/y ranges
def crop(image, ymin, ymax, xmin, xmax):
    return image[ymin:ymax, xmin:xmax]


def calc_combined_coordinates(base_x, base_y, offset_x, offset_y, base_rotate, base_x_end):
    if base_rotate:
        result_x = base_x_end - cube_size - offset_y # 250==CUBE_SIZE
        result_y = base_y + offset_x
    else:
        result_x = base_x + offset_x
        result_y = base_y + offset_y
    return result_x, result_y

def load_img_for_name(file_name):
    img_path = ""
    if "-" in file_name:
        name_parts = file_name.split("-")
        unique_part = name_parts[0] + "-" + name_parts[1] + "-" + name_parts[2]
        img_path = imgs_root + name_parts[0] + "/" + \
            unique_part + "/"
    else:
        img_path = imgs_root + file_name[0:4] + "/" + \
            file_name[0:4] + "-" + file_name[4:9] + "-V/"
    
    for file_ in list_blobs_with_prefix(img_path):
        if (" _018" in file_):        
            return gcp_img_read(file_) # img.imread(img_path + file_)

def load_cropped_for_name(file_name):
    img_path = cropped_root + "/" + file_name + " _018.jpg.npy"
    return gcp_np_load(img_path) # np.load(img_path)

def load_front_for_name(file_name):
    img_path = ""
    if "-" in file_name:
        front_file_name = file_name.replace("-V-","-R-")
        name_parts = front_file_name.split("-")
        unique_part = name_parts[0] + "-" + name_parts[1] + "-" + name_parts[2]
        img_path = imgs_root + name_parts[0] + "/" + \
            unique_part + "/"
    else:
        img_path = imgs_root + file_name[0:4] + "/" + \
            file_name[0:4] + "-" + file_name[4:9] + "-R/"
    #import pdb; pdb.set_trace()
    #for root, dirs, files in os.walk(img_path):
    for file_ in list_blobs_with_prefix(img_path):
        if (" _018" in file_):        
            return gcp_img_read(file_) # img.imread(img_path + file_)
    
# Pre-process the validation set
def folder_walker(path, filter_text=""):
    result = []
    #for root, dirs, files in os.walk(img_path):
    for file_ in list_blobs_with_prefix(path):
        if not file_.startswith("."):
            if (filter_text == "" or filter_text in file_):
                result.append(file_)
    return result

rotate_raw = folder_walker(all_root + "front_rotate/", "-R-")
rotate = []
for file_ in rotate_raw:
    file_ = file_[file_.rfind('/')+1:]
    split = file_.split("-")
    rotate.append(split[0]+split[1])
    

In [3]:
# RUN Utility functions

# Simple crop by x/y ranges
def crop(image, ymin, ymax, xmin, xmax):
    return image[ymin:ymax, xmin:xmax]

# returns a logical matrix of values beyond a threshld
def thresholded(image, val): 
    return np.logical_and(*[image[...] > val  for t in enumerate([0, 0])])

def find_min_max_without_orphand_pixels(nonzero_dimension, crop_filter=20):
    sorted = np.sort(nonzero_dimension)
    prev=-1
    min_val = sorted[0]
    for i, x in enumerate(sorted[:100]):
        if prev >= 0 and x - prev > crop_filter:
            min_val = x
        prev = x
    prev=-1
    max_val = sorted[-1]
    for i, x in enumerate(sorted[-100:]):
        if prev >= 0 and x - prev > crop_filter:
            max_val = prev
            break
        prev = x
    
    return min_val, max_val

def calc_min_max_coordinates(image, crop_val=50):
    temp = thresholded(image, crop_val)
    temp = temp * 1
    temp = np.nonzero(temp)
    ymin, ymax = find_min_max_without_orphand_pixels(temp[0])
    xmin,xmax = find_min_max_without_orphand_pixels(temp[1])
    return ymin, ymax, xmin, xmax

def calc_min_max_coordinates_dynamic(image, cutoff=1):
    temp = exposure.equalize_adapthist(image, clip_limit=0.03)
    flat = np.sort(np.matrix.getA1(temp))
    sum_all = np.sum(flat)
    index = np.argmin(flat.cumsum() < (sum_all * cutoff))

    temp = thresholded(temp, flat[index])
    temp = temp * 1
    temp = np.nonzero(temp)
    ymin, ymax = find_min_max_without_orphand_pixels(temp[0])
    xmin,xmax = find_min_max_without_orphand_pixels(temp[1])
    return ymin, ymax, xmin, xmax

# initial static crop and a seondary dynamic crop based on signal2noise ratio
def crop_full_scan(image, x_start, x_end, y_start, y_end):
    temp = crop(image, y_start, y_end, x_start, x_end)
    ymin, ymax, xmin, xmax = calc_min_max_coordinates_dynamic(temp, cutoff=BG_2_OBJ_RATIO)
    temp = crop(image, y_start+ymin, y_start+ymax, x_start+xmin, x_start+xmax)
    return temp

def crop_thresholded(image):
    temp = crop(image, 0, image.shape[0]-1, 0, image.shape[1]-1)
    ymin, ymax, xmin, xmax = calc_min_max_coordinates(temp)
    temp = crop(image, ymin, ymax, xmin, xmax)
    return temp

def read_and_crop(image_name, x_start=X_START, x_end=X_END, y_start=Y_START, y_end=Y_END):
    if "il239838" in os.getcwd():
        image = img.imread(ROOT_FOLDER + image_name)
    else:
        f = urllib.request.urlopen("https://dl.dropboxusercontent.com/s/31b96942qdcn73k/" + image_name)
        image = img.imread(f, format='jpeg')

    # Smart-crop the image to get rid of all the noise and redundant area
    # return crop_full_scan(image)
    cropped = crop_full_scan(image, x_start, x_end, y_start, y_end)
    return exposure.equalize_adapthist(cropped, clip_limit=0.03)


# TODO: fix performance!!! http://scikit-image.org/docs/dev/user_guide/tutorial_parallelization.html
def combine_3_images_to_RGB(red, green, blue):
    new_image = np.empty((blue.shape[0],blue.shape[1],3))
    for x in range(0, blue.shape[0]):
        for y in range(0, blue.shape[1]):
            new_image[x,y,0] = red[x,y]
            new_image[x,y,1] = green[x,y]
            new_image[x,y,2] = blue[x,y]
    return new_image

def slice_image_left_edge(original, width=200, rotate=0):
    rot = ndimage.rotate(original, rotate)
    # Slice the left slice of the so-called "blue" image
    left_edge_orig = crop(rot, 1, 1400, 1, width)
    left_edge_orig = crop_thresholded(left_edge_orig)

    # Copy to a new array so we don't thrash the origin
    left_edge = np.empty_like (left_edge_orig)
    np.copyto(left_edge, left_edge_orig)

    # Zero down low level "noise" values
    low_values_indices = left_edge < 30  # Where values are low
    left_edge[low_values_indices] = 0  # All low values set to 0
    return left_edge

def get_best_angle_rotation(original, crop=True, width=200):
    min_var = 99999999999
    best_angle = -10
    for x in range(-5,5):
        if crop:            
            rot_edge = slice_image_left_edge(original, width, x)
        else:
            rot_edge = ndimage.rotate(original, x)
        left_var = np.var(rot_edge, axis=1)
        # left_var = np.apply_along_axis(lambda v: np.var(v[np.nonzero(v)]), 1, rot_edge)
        var_sum = np.sum(left_var)
        if (var_sum < min_var):
            min_var = var_sum
            best_angle = x
    print ("best_angle="+str(best_angle))
    return best_angle

#     import pdb; pdb.set_trace()
def calc_neighbors(slice_map, col, row):
    # import pdb; pdb.set_trace()
    if ((col-1, row) in slice_map and slice_map[(col-1, row)] != None):
        slice_map[(col, row)]["left"] = slice_map[(col-1, row)]
        slice_map[(col-1, row)]["right"] = slice_map[(col, row)]
    if ((col+1, row) in slice_map and slice_map[(col+1, row)] != None):
        slice_map[(col, row)]["right"] = slice_map[(col+1, row)]
        slice_map[(col+1, row)]["left"] = slice_map[(col, row)]
    if ((col, row-1) in slice_map and slice_map[(col, row-1)] != None):
        slice_map[(col, row)]["top"] = slice_map[(col, row-1)]
        slice_map[(col, row-1)]["bottom"] = slice_map[(col, row)]
    if ((col, row+1) in slice_map and slice_map[(col, row+1)] != None):
        slice_map[(col, row)]["bottom"] = slice_map[(col, row+1)]
        slice_map[(col, row+1)]["top"] = slice_map[(col, row)]
    


def VAL_create_cube(name, raw, x, y):
    cube = {}
    cube["cube"] = raw
    cube["file"] = name
    if name.find('P') == 0 and name.find('Fg') > 0:
        cube["index"] = int(name[name.find('P')+1:name.find('P')+4]) * 1000 + int(name[name.find('Fg')+2:name.find('Fg')+5])
    else:
        print("Found a ZERO index cube with the name:"+name)
        cube["index"] = 0
    cube["top_row"] = x
    cube["left_col"] = y
    cube["right_col"] = y + CUBE_SIZE
    return cube
    

ZERO_CUBE = VAL_create_cube("ZERO", np.zeros((CUBE_SIZE, CUBE_SIZE), dtype=np.int), -1, -2)

# slice an image to cubes with 250X250 pixel size
def VAL_slice_TEAR_to_static_slices(name, cropped_original):
    structure = {}
    # cropped_original = cropped_original / 256 # divide by 256 to "normalize" between 0 and 1

    # import pdb; pdb.set_trace()
    x, y = cropped_original["cut"].shape
    # print (x,y)
    n = 0
    # see n offset to see offset in pixels on the x axis == rows. every n equals CUBE_SIZE
    while ((n + 1) * CUBE_SIZE < x):
        # mark the piece as narrow so the first would be counted as lastt too
        narrow = True if ((CUBE_SIZE + (4 * EDGE_GAP)) > y) else False
        # cut a cube of 250X250 at the FIRST column
        start_row_px = int(np.round(n * CUBE_SIZE, -1))
        end_row_px = int(np.round((n + 1) * CUBE_SIZE, -1))
        cube = (crop(cropped_original["cut"], start_row_px, end_row_px, EDGE_GAP, CUBE_SIZE + EDGE_GAP))
        # keep only cubes for which half of the pixels have some "color"
        if np.median(cube) > 0.2: # aligned with the normalization 0.2 correlates to 50
            # keep the cube
            new_cube = VAL_create_cube(name, cube, start_row_px, EDGE_GAP)
            new_cube["col"] = 0 # marks that the cube is on the first col of the piece
            new_cube["row"] = n
            new_cube["last"] = narrow # marks that the cube is on the last col of the piece
            new_cube["orig"] = cropped_original
            new_cube["col_px_left"] = cropped_original["col_px"] + EDGE_GAP
            new_cube["col_px_right"] = cropped_original["col_px"] + CUBE_SIZE + EDGE_GAP
            new_cube["row_px_top"] = cropped_original["row_px"] + start_row_px
            new_cube["row_px_bottom"] = cropped_original["row_px"] + end_row_px
            structure[(0, n)] = new_cube

        # cut a cube of 250X250 at the LAST column
        cube = (crop(cropped_original["cut"], start_row_px, end_row_px, y - CUBE_SIZE - EDGE_GAP, y - EDGE_GAP))
        # keep only cubes for which half of the pixels have some "color"
        # aligned with the normalization 0.2 correlates to 50
        if np.median(cube) > 0.2:
            # keep the cube
            new_cube = VAL_create_cube(name, cube, start_row_px, y - CUBE_SIZE - EDGE_GAP)
            new_cube["col"] = 1 # marks that the cube is on the last col of the piece
            new_cube["row"] = n
            new_cube["last"] = not narrow # like col - marks that the cube is on the last col of the piece
            new_cube["orig"] = cropped_original
            new_cube["col_px_left"] = cropped_original["col_px"] + y - CUBE_SIZE - EDGE_GAP
            new_cube["col_px_right"] = cropped_original["col_px"] + y - EDGE_GAP
            new_cube["row_px_top"] = cropped_original["row_px"] + start_row_px
            new_cube["row_px_bottom"] = cropped_original["row_px"] + end_row_px
            structure[(1, n)] = new_cube

    #         m = 0
    #         # every 250 pixels on the y axis == cols
    #         while ((m + 1) * CUBE_SIZE < y):            
    #             if ((m == 0) or ((m + 2) * CUBE_SIZE >= y)): # Only keep the left and right edges of the piece for matching!!
    #                 # cut a cube of 250X250
    #                 cube = crop(cropped_original["cut"], n * CUBE_SIZE, (n + 1) * CUBE_SIZE, m * CUBE_SIZE, (m + 1) * CUBE_SIZE)
    #                 # keep only cubes for which half of the pixels have some "color"
    #                 # print(np.median(cube))
    #                 if np.median(cube) > 0.2: # aligned with the normalization 0.2 correlates to 50
    #                     # keep the cube
    #                     new_cube = VAL_create_cube(name, cube, n * CUBE_SIZE, m * CUBE_SIZE)
    #                     new_cube["col"] = m
    #                     new_cube["row"] = n
    #                     new_cube["orig"] = cropped_original
    #                     new_cube["col_px_left"] = cropped_original["col_px"] + m * CUBE_SIZE
    #                     new_cube["col_px_right"] = cropped_original["col_px"] + (m + 1) * CUBE_SIZE
    #                     new_cube["row_px_top"] = cropped_original["row_px"] + n * CUBE_SIZE
    #                     new_cube["row_px_bottom"] = cropped_original["row_px"] + (n + 1) * CUBE_SIZE
    #                     if ((m + 2) * CUBE_SIZE >= y):
    #                         new_cube["last"] = True
    #                     else:
    #                         new_cube["last"] = False
    #                     structure[(m, n)] = new_cube
    #             m += 1
        n += 0.2 # currently set to jump in 50 px offset
            
    # this loop has to be performed only after we've established all the None cubes
    for cube in structure.values():
        # set the reference to neighbor cubes
        if cube != None:
            calc_neighbors(structure, cube["col"], cube["row"])

    # return the data structure with all the cubes and the counters of the rows and columns
    return structure.values()

#import pdb; pdb.set_trace()
def VAL_slice_to_static_slices(name, cropped_original, img_data):
    structure = {}
    # cropped_original = cropped_original / 256 # divide by 256 to "normalize" between 0 and 1

    # import pdb; pdb.set_trace()
    x, y = cropped_original.shape
    # print (x,y)
    n = 0
    # see n offset to see offset in pixels on the x axis == rows. every n equals CUBE_SIZE
    while ((n + 1) * CUBE_SIZE < x):
        # mark the piece as narrow so the first would be counted as lastt too
        narrow = True if ((CUBE_SIZE + (4 * EDGE_GAP)) > y) else False
        very_narrow = True if ((CUBE_SIZE + EDGE_GAP) > y) else False
        # cut a cube of 250X250 at the FIRST column
        start_row_px = int(np.round(n * CUBE_SIZE, -1))
        end_row_px = int(np.round((n + 1) * CUBE_SIZE, -1))

        # crop a cube on the starting edge of the current row. 
        # start with EDGE_GAP and move inwards untill we have value
        fg_value = 0
        iteration = 0 if very_narrow else 1 # patch... covers most cases... but not all
        while fg_value <= 0.2 and iteration <= 5:
            cube = (crop(cropped_original, start_row_px, end_row_px, \
                         EDGE_GAP * iteration, CUBE_SIZE + (EDGE_GAP * iteration)))
            iteration += 1
            fg_value = np.median(cube)
            
        # keep only cubes for which half of the pixels have some "color"
        # aligned with the normalization 0.2 correlates to 50
        # discard cubes which are too narrow - very narrow slices
        if np.median(cube) > 0.2 and cube.shape[1] == 250: 
            # keep the cube
            new_cube = VAL_create_cube(name, cube, start_row_px, EDGE_GAP)
            new_cube["col"] = 0 # marks that the cube is on the first col of the piece
            new_cube["row"] = n
            new_cube["last"] = narrow # marks that the cube is on the last col of the piece
            new_cube["orig"] = cropped_original
            new_cube["col_px_left"] = img_data["col_px"] + (EDGE_GAP * iteration)
            new_cube["col_px_right"] = img_data["col_px"] + CUBE_SIZE + (EDGE_GAP * iteration)
            new_cube["row_px_top"] = img_data["row_px"] + start_row_px
            new_cube["row_px_bottom"] = img_data["row_px"] + end_row_px
            structure[(0, n)] = new_cube

        # crop a cube on the starting edge of the current row. 
        # start with EDGE_GAP and move inwards untill we have value
        fg_value = 0
        iteration = 0 if very_narrow else 1 # patch... covers most cases... but not all
        while fg_value <= 0.2 and iteration <= 5:
            cube = (crop(cropped_original, start_row_px, end_row_px, \
                         y - CUBE_SIZE - (EDGE_GAP * iteration), y - (EDGE_GAP * iteration)))
            iteration += 1
            fg_value = np.median(cube)
        
        
        # keep only cubes for which half of the pixels have some "color"
        # aligned with the normalization 0.2 correlates to 50
        # discard cubes which are too narrow - very narrow sices
        if np.median(cube) > 0.2 and cube.shape[1] == 250:
            # keep the cube
            new_cube = VAL_create_cube(name, cube, start_row_px, y - CUBE_SIZE - EDGE_GAP)
            new_cube["col"] = 1 # marks that the cube is on the last col of the piece
            new_cube["row"] = n
            new_cube["last"] = not narrow # like col - marks that the cube is on the last col of the piece
            new_cube["orig"] = cropped_original
            new_cube["col_px_left"] = img_data["col_px"] + y - CUBE_SIZE - (EDGE_GAP * iteration)
            new_cube["col_px_right"] = img_data["col_px"] + y - (EDGE_GAP * iteration)
            new_cube["row_px_top"] = img_data["row_px"] + start_row_px
            new_cube["row_px_bottom"] = img_data["row_px"] + end_row_px
            structure[(1, n)] = new_cube

        n += 0.2 # currently set to jump in 50 px offset
            
    ## this loop has to be performed only after we've established all the None cubes
    #for cube in structure.values():
    #    # set the reference to neighbor cubes
    #    if cube != None:
    #        calc_neighbors(structure, cube["col"], cube["row"])

    # return the data structure with all the cubes and the counters of the rows and columns
    return structure.values()

def pad_above(original, above, amount):
    res = np.insert(original["cube"], np.zeros(amount), above["cube"][-amount:], axis=0)
    res = np.delete(res, np.arange(CUBE_SIZE,CUBE_SIZE+amount), axis=0)
    cube = VAL_create_cube(original["file"], res, original["top_row"] - amount, original["left_col"])
    cube["col_px_left"] = original["col_px_left"]
    cube["col_px_right"] = original["col_px_right"]
    cube["row_px_top"] = original["row_px_top"] - amount
    cube["row_px_bottom"] = original["row_px_bottom"] - amount
    return cube

def pad_below(original, below, amount):
    res = np.insert(original["cube"], np.full(amount, CUBE_SIZE), below["cube"][:amount], axis=0)
    res = np.delete(res, np.arange(0, amount), axis=0)
    cube =  VAL_create_cube(original["file"], res, original["top_row"] + amount, original["left_col"])
    cube["col_px_left"] = original["col_px_left"]
    cube["col_px_right"] = original["col_px_right"]
    cube["row_px_top"] = original["row_px_top"] + amount
    cube["row_px_bottom"] = original["row_px_bottom"] + amount
    return cube
    
    
def pad_left(original, left, amount):
    res = np.insert(original["cube"], np.zeros(amount, dtype=int), left["cube"][:,-amount:], axis=1)
    res = np.delete(res, np.arange(CUBE_SIZE, CUBE_SIZE+amount), axis=1)
    cube = VAL_create_cube(original["file"], res, original["top_row"], original["left_col"] - amount)
    cube["col_px_left"] = original["col_px_left"] - amount
    cube["col_px_right"] = original["col_px_right"] - amount
    cube["row_px_top"] = original["row_px_top"]
    cube["row_px_bottom"] = original["row_px_bottom"]
    return cube

def pad_right(original, right, amount):
    res = np.insert(original["cube"], [CUBE_SIZE], right["cube"][:,:amount], axis=1)
    res = np.delete(res, np.arange(0, amount), axis=1)
    cube = VAL_create_cube(original["file"], res, original["top_row"], original["left_col"] + amount)
    cube["col_px_left"] = original["col_px_left"] + amount
    cube["col_px_right"] = original["col_px_right"] + amount
    cube["row_px_top"] = original["row_px_top"]
    cube["row_px_bottom"] = original["row_px_bottom"]
    return cube
    

# "Shave" the right edge of the cube with <gap> pixels and pad with zeros on the left
def shave_right(original, amount):
    return pad_left(original, ZERO_CUBE, amount)
    

# "Shave" the left edge of the cube with <gap> pixels and pad with zeros on the right    
def shave_left(original, amount):
    return pad_right(original, ZERO_CUBE, amount)
    

# concatenate cubes 
def concatenate_cubes(left, right, slice_size):
    con = np.concatenate((left["cube"][:,-slice_size:], right["cube"][:,:slice_size]), axis=1)
    x_delta = right["top_row"] - left["top_row"]
    y_delta = right["left_col"] - left["right_col"] 
    return con, x_delta, y_delta

# concatenate cubes 
def VAL_concatenate_cubes(left, right, slice_size):
    right_img = right["cube"]
    # next block is not relevant for training ...
    #     # if the left cube is matched to another left cube (or right cube to another right cube) then rotate the right
    #     # cube by 180 so we try to match it upside down, covering the option that the cube was pictured rotated
    #     if ((left["col"] == 0 and right["col"] == 0) or (left["col"] != 0 and right["col"] != 0)):
    #         right_img = np.rot90(right["cube"], 2);

    con = np.concatenate((left["cube"][:,-slice_size:], right_img[:,:slice_size]), axis=1)

    # next block calculates distance based on the distance between left's right-top corner and right's left-top corner    
    #     x_delta = right["top_row"] - left["top_row"]
    #     y_delta = right["left_col"] - left["right_col"] 

    # next block calculates the distance between the centers of cubes, accounting for test set's possibility of reverse slices (left instead of right and vice versa)
    x_delta = right["row_px_top"] - left["row_px_top"] # equivalent to distance between vertical centers
    y_delta = (right["col_px_left"] + (slice_size / 2)) - (left["col_px_right"] - (slice_size / 2)) # measuring the distance between horizontal centers of the slices

    return con, x_delta, y_delta, left["file"], right["file"]
    

# concatenate cubes while artificially creating a gap between them. Pad the other end of the cube with zeros
def concatenate_cubes_zero_pad_gaps(left_orig, right_orig, gap):
    left = left_orig if gap == 0 else shave_right(left_orig, gap)
    right = right_orig if gap == 0 else shave_left(right_orig, gap)
    return concatenate_cubes(left, right)    

# concatenate cubes while artificially creating a gap between them. Pad the other end of the cobe with the nearby
# continuation of the cubes
def concatenate_cubes_with_gap(left_orig, right_orig, gap, left_pad, right_pad, slice_size):
    # import pdb; pdb.set_trace()
    left = left_orig if gap == 0 else pad_left(left_orig, left_pad, gap)
    right = right_orig if gap == 0 else pad_right(right_orig, right_pad, gap)
    return concatenate_cubes(left, right, slice_size)        

# convert the data structure of cubes into a train set of 2 arrays of images and labels
# each image is a concatanation of 2 images from the original cubes set, covering all combinations of images
# effectively creating Nx(N-1) images
def VAL_build_train_set_for_euclidean_distance(cubes, slice_size, folder):
    # clean folder before starting
    for root, dirs, files in os.walk(folder):
        for f in files:
            os.unlink(os.path.join(root, f))

    #import pdb; pdb.set_trace()
    warnings.filterwarnings("ignore")

    train_imgs = []
    train_x_delta = []
    train_y_delta = []
    train_left_obj = []
    train_right_obj = []
    # iterate over all cubes   
    for curr in cubes:
        # iterate over the others (effectively n^2)
        for adj in cubes:
            if (adj["file"] != curr["file"]): # no need to test against self CURRENTLY checking from directions!!!
                #import pdb; pdb.set_trace()
                # append the adjacent image to the current image
                conc, x_delta, y_delta, x_file, y_file = VAL_concatenate_cubes(curr, adj, slice_size)
                output = folder+x_file+"_"+str(curr["top_row"])+"_"+str(curr["left_col"])+"---"+y_file+"_"+str(adj["top_row"])+"_"+str(adj["left_col"])
                np.save(output, conc)
                train_imgs.append(output)
                train_x_delta.append(x_delta)
                train_y_delta.append(y_delta)
                train_left_obj.append(curr)
                train_right_obj.append(adj)

    warnings.filterwarnings("default")
    return train_imgs, train_x_delta, train_y_delta, train_left_obj, train_right_obj


# convert the data structure of cubes into a train set of 2 arrays of images and labels
# each image is a concatanation of 2 images from the original cubes set, covering all combinations of images
# effectively creating Nx(N-1) images
def ORIG_build_train_set(cubes, gap):
    # import pdb; pdb.set_trace()
    warnings.filterwarnings("ignore")

    train_imgs = []
    train_lbls = []
    train_x_delta = []
    train_y_delta = []
    # iterate over the rows and cols, essentially going over the grid of sliced cubes
    for row in range(0, rows):
        for col in range(0, cols):
            # if this cube exists (could have been removed previously due to lack of data)
            if (cubes[(col, row)] != None):
                # for each "current" image in the iteration
                curr = cubes[(col, row)]
                # iterate over all the cubes to find all the "other" (adjacent) cubes
                for adj_row in range(0, rows):
                    for adj_col in range(0, cols):
                        if (adj_row != row or adj_col != col):
                            if (cubes[(adj_col, adj_row)] != None):
                                adj = cubes[(adj_col, adj_row)]
                                # append the adjacent image to the current image
                                # pass the filling cubes on the right and left to pad against the gap
                                if (gap == 0 or ("left" in curr.keys() and "right" in adj.keys())):
                                    if (gap == 0):
                                        conc, x_delta, y_delta = concatenate_cubes(curr, adj, slice_size)
                                    else:
                                        conc, x_delta, y_delta = concatenate_cubes_with_gap(curr, adj, gap, curr["left"], adj["right"], slice_size)
                                    train_imgs.append(conc)
                                    train_x_delta.append(x_delta)
                                    train_y_delta.append(y_delta)
                                    # if the adj image is on the same row and on the right of the curr image - it will be marked as match    
                                    if (adj_row == row and adj_col == (col + 1)):
                                        # mark the image as matched
                                        train_lbls.append([0,1])
                                        # need to enrich the set with a few more tru positive samples - so we offset 
                                        # the matched images up ad down a few times and create more matches
                                        if ("top" in curr.keys() and "top"in adj.keys()):
                                            for i in range(5, 101, 5):
                                                curr1 = pad_above(curr, curr["top"],i)
                                                adj1 = pad_above(adj, adj["top"],i)
                                                if (gap == 0 or ("left" in curr.keys() and "right" in adj.keys() and "top" in curr["left"].keys() and "top"in curr["right"].keys())):
                                                    if (gap == 0):
                                                        conc, x_delta, y_delta = concatenate_cubes(curr1, adj1, slice_size)
                                                    else:
                                                        curr1Left = pad_above(curr["left"], curr["left"]["top"], i) # FIXIT?
                                                        adj1Right = pad_above(adj["right"], curr["right"]["top"], i) # FIXIT?
                                                        conc, x_delta, y_delta = concatenate_cubes_with_gap(curr1, adj1, gap, curr1Left, adj1Right, slice_size)
                                                    train_imgs.append(conc)
                                                    train_x_delta.append(x_delta)
                                                    train_y_delta.append(y_delta)
                                                    # mark the image as matched
                                                    train_lbls.append([0,1])
                                        if ("bottom" in curr.keys() and "bottom"in adj.keys()):
                                            for i in range(5, 101, 5):
                                                curr1 = pad_below(curr, curr["bottom"],i)
                                                adj1 = pad_below(adj, adj["bottom"],i)
                                                if (gap == 0 or ("left" in curr.keys() and "right" in adj.keys() and "bottom" in curr["left"].keys() and "bottom"in curr["right"].keys())):
                                                    if (gap == 0):
                                                        conc, x_delta, y_delta = concatenate_cubes(curr1, adj1, slice_size)
                                                    else:
                                                        curr1Left = pad_below(curr["left"], curr["left"]["bottom"], i) # FIXIT?
                                                        adj1Right = pad_below(adj["right"], curr["right"]["bottom"], i) # FIXIT?
                                                        conc, x_delta, y_delta = concatenate_cubes_with_gap(curr1, adj1, gap, curr1Left, adj1Right, slice_size)
                                                    train_imgs.append(conc)
                                                    train_x_delta.append(x_delta)
                                                    train_y_delta.append(y_delta)
                                                    # mark the image as matched
                                                    train_lbls.append([0,1])
                                        if ("left" in curr.keys()): # enough to check only the curr as the left of the adj is the curr
                                            for i in range(5, 101, 5):
                                                curr1 = pad_left(curr, curr["left"],i)
                                                adj1 = pad_left(adj, adj["left"],i) # essentially the curr
                                                if (gap == 0 or ("left" in curr.keys() and "right" in adj.keys())):
                                                    if (gap == 0):
                                                        conc, x_delta, y_delta = concatenate_cubes(curr1, adj1, slice_size)
                                                    else:
                                                        curr1Left = pad_left(curr["left"], ZERO_CUBE, i) # FIXIT? + assuming the gap will not be more than 150
                                                        adj1Right = pad_left(adj["right"], ZERO_CUBE, i) # FIXIT? + assuming the gap will not be more than 150
                                                        conc, x_delta, y_delta = concatenate_cubes_with_gap(curr1, adj1, gap, curr1Left, adj1Right, slice_size)
                                                    train_imgs.append(conc)
                                                    train_x_delta.append(x_delta)
                                                    train_y_delta.append(y_delta)
                                                    # mark the image as matched
                                                    train_lbls.append([0,1])
                                        if ("right" in adj.keys()): # enough to check only the adj as the right of the curr is the adj
                                            for i in range(5, 101, 5):
                                                curr1 = pad_right(curr, curr["right"],i) # essentially the adj
                                                adj1 = pad_right(adj, adj["right"],i)
                                                if (gap == 0 or ("left" in curr.keys() and "right" in adj.keys())):
                                                    if (gap == 0):
                                                        conc, x_delta, y_delta = concatenate_cubes(curr1, adj1, slice_size)
                                                    else:
                                                        curr1Left = pad_right(curr["left"], ZERO_CUBE, i) # FIXIT? + assuming the gap will not be more than 150
                                                        adj1Right = pad_right(adj["right"], ZERO_CUBE, i) # FIXIT? + assuming the gap will not be more than 150
                                                        conc, x_delta, y_delta = concatenate_cubes_with_gap(curr1, adj1, gap, curr1Left, adj1Right, slice_size)
                                                    train_imgs.append(conc)
                                                    train_x_delta.append(x_delta)
                                                    train_y_delta.append(y_delta)
                                                    # mark the image as matched
                                                    train_lbls.append([0,1])
                                    else:
                                        # mark the image as not matched
                                        train_lbls.append([1,0])
                                
    warnings.filterwarnings("default")
    return train_imgs, train_lbls, train_x_delta, train_y_delta

Found a ZERO index cube with the name:ZERO


In [4]:
# RUN Utility functions 2
SAVE_PNG=False
def save_img(path, img):
    np.save(path, img)  
    if SAVE_PNG:
        plt.imsave(path+".png", img, cmap=plt.cm.gray)
                    
def VAL_add_tolerance_matches(slice_size, folder, train_imgs, train_lbls, train_x_delta, 
                              train_y_delta, curr, adj, tolerance_factor=0):
    # need to enhance the set with a few more true positive samples
    # allowing some up and down tolerance
    if ("top" in curr.keys()):
        for i in range(0, tolerance_factor * 10, 10):
            if i == 0:
                continue
            curr1 = pad_above(curr, curr["top"],i)
            adj1 = adj
            conc, x_delta, y_delta, x_file, y_file = VAL_concatenate_cubes(curr1, adj1, slice_size)
            output = folder+"1="+x_file+"_"+str(curr1["top_row"])+"_"+str(curr1["left_col"])+"---"+y_file+"_"+str(adj1["top_row"])+"_"+str(adj1["left_col"])
            # print(">>> MATCH >>>"+output)
            save_img(output, conc)
            # print(">>> >>> >>> SAVED")
            train_imgs.append(output)
            train_x_delta.append(x_delta)
            train_y_delta.append(y_delta)
            # mark the image as matched
            train_lbls.append([0,1])
    if ("top" in adj.keys()):
        for i in range(0, tolerance_factor * 10, 10):
            if i == 0:
                continue
            curr1 = curr
            adj1 = pad_above(adj, adj["top"],i)
            conc, x_delta, y_delta, x_file, y_file = VAL_concatenate_cubes(curr1, adj1, slice_size)
            output = folder+"1="+x_file+"_"+str(curr1["top_row"])+"_"+str(curr1["left_col"])+"---"+y_file+"_"+str(adj1["top_row"])+"_"+str(adj1["left_col"])
            # print(">>> MATCH >>>"+output)
            save_img(output, conc)
            # print(">>> >>> >>> SAVED")
            train_imgs.append(output)
            train_x_delta.append(x_delta)
            train_y_delta.append(y_delta)
            # mark the image as matched
            train_lbls.append([0,1])
    if ("bottom" in curr.keys()):
        for i in range(0, tolerance_factor * 10, 10):
            if i == 0:
                continue
            curr1 = pad_below(curr, curr["bottom"],i)
            adj1 = adj
            conc, x_delta, y_delta, x_file, y_file = VAL_concatenate_cubes(curr1, adj1, slice_size)
            output = folder+"1="+x_file+"_"+str(curr1["top_row"])+"_"+str(curr1["left_col"])+"---"+y_file+"_"+str(adj1["top_row"])+"_"+str(adj1["left_col"])
            # print(">>> MATCH >>>"+output)
            save_img(output, conc)
            # print(">>> >>> >>> SAVED")
            train_imgs.append(output)
            train_x_delta.append(x_delta)
            train_y_delta.append(y_delta)
            # mark the image as matched
            train_lbls.append([0,1])
    if ("bottom"in adj.keys()):
        for i in range(0, tolerance_factor * 10, 10):
            if i == 0:
                continue
            curr1 = curr
            adj1 = pad_below(adj, adj["bottom"],i)
            conc, x_delta, y_delta, x_file, y_file = VAL_concatenate_cubes(curr1, adj1, slice_size)
            output = folder+"1="+x_file+"_"+str(curr1["top_row"])+"_"+str(curr1["left_col"])+"---"+y_file+"_"+str(adj1["top_row"])+"_"+str(adj1["left_col"])
            # print(">>> MATCH >>>"+output)
            save_img(output, conc)
            # print(">>> >>> >>> SAVED")
            train_imgs.append(output)
            train_x_delta.append(x_delta)
            train_y_delta.append(y_delta)
            # mark the image as matched
            train_lbls.append([0,1])

                            
                
# IMPORTANT: enrich_factor determines how many "duplications" of TRUE values will we have in the train set
# This allows for a more balanced train set however, it reduces the strictness of the matches 
# i.e. (not sure why) when we have multiple nearby "duplicates" matches we get much more matches in the validation
# PARAMS: enrich_factor=1 means no enrich/duplicate, 20 means duplicate by 20, every 10 pixels
# PARAMS: tolerance_factor=0 means only match against exact horizon, each notch equals additional 10 pixels tolerance
def NEW_build_train_set_for_binary_labeling(cubes, slice_size, folder, enrich_factor=1, tolerance_factor=0): 
    # enrich_factor is split by 2 because it is dual-sided and 1 means actually no enrichment - i.e. 0.5
    enrich_factor = enrich_factor / 2 
    # clean folder before starting
    for root, dirs, files in os.walk(folder):
        for f in files:
            os.unlink(os.path.join(root, f))

    warnings.filterwarnings("ignore")

    train_imgs = []
    train_lbls = []
    train_x_delta = []
    train_y_delta = []
    discard_c = 0

    # import pdb; pdb.set_trace()

    # iterate over the cubes
    for curr in cubes:
        # iterate over the others (effectively n^2)
        for adj in cubes:
            # Initial filter: what CAN be matched against what?
            # 1 - not of the same fragment (file==fragment)
            # 2 - they ARE of the same tear - don't want to confuse the learning with false data coming from different tears
            # 3 - no need to test against self and avoid checking from both directions
            if  adj["file"] != curr["file"] and \
                adj["tear"] == curr["tear"] and \
                curr["piece_col"] < adj["piece_col"]: 
                # last condition above - actually ignores pieces of the same col but different rows
                # the assumption is that they are either "not-match" and then will tilt the balance further to not-match
                # or they are "somewhat-matching" but in a way that might confuse the algorithm
                
                # print(">>> >>>"+str(curr["cube"].shape)+" <<< <<<"+str(adj["cube"].shape))
                # append the adjacent image to the current image
                conc, x_delta, y_delta, x_file, y_file = VAL_concatenate_cubes(curr, adj, slice_size)

                train_x_delta.append(x_delta)
                train_y_delta.append(y_delta)
                                
                # Condition for marking as match:
                # 1 - the adj piece is on the same row as the curr
                # 2 - the adj piece is just to the right of the curr
                # 3 - the curr cube is on the right edge of the piece
                # 4 - the adj cube is on the left edge of the piece
                # 5 - the cubes are in the same horizon
                if  curr["piece_row"] == adj["piece_row"] and \
                    curr["piece_col"] + 1 == adj["piece_col"] and \
                    (curr["col"] != 0 or curr["last"]) and \
                    (adj["col"] == 0 or not adj["last"]) and \
                    np.abs(x_delta) < 50: 

                    # print(x_delta, y_delta)
                    
                    # mark the image as matched
                    output = folder+"1="+x_file+"_"+str(curr["top_row"])+"_"+str(curr["left_col"])+"---"+y_file+"_"+str(adj["top_row"])+"_"+str(adj["left_col"])
                    # print(">>> MATCH >>>"+output)
                    save_img(output, conc)
                    # print(">>> >>> >>> SAVED")
                    train_imgs.append(output)
                    train_lbls.append([0,1])

                    #import pdb; pdb.set_trace()
                    # TOLERANCE
                    VAL_add_tolerance_matches(slice_size, folder, train_imgs, train_lbls, train_x_delta, 
                                              train_y_delta, curr, adj, tolerance_factor)
                    
                    # ENRICH/DUPLICATE
                    # need to enrich the set with a few more true positive samples - so we offset 
                    # the matched images up and down a few times and create more matches
                    if ("top" in curr.keys() and "top" in adj.keys()):
                        for i in range(0, 121, int(120/enrich_factor)):
                            if i == 0:
                                continue
                            curr1 = pad_above(curr, curr["top"],i)
                            adj1 = pad_above(adj, adj["top"],i)
                            conc, x_delta, y_delta, x_file, y_file = VAL_concatenate_cubes(curr1, adj1, slice_size)
                            output = folder+"1="+x_file+"_"+str(curr1["top_row"])+"_"+str(curr1["left_col"])+"---"+y_file+"_"+str(adj1["top_row"])+"_"+str(adj1["left_col"])
                            # print(">>> MATCH >>>"+output)
                            save_img(output, conc)
                            # print(">>> >>> >>> SAVED")
                            train_imgs.append(output)
                            train_x_delta.append(x_delta)
                            train_y_delta.append(y_delta)
                            # mark the image as matched
                            train_lbls.append([0,1])

                            # TOLERANCE
                            VAL_add_tolerance_matches(slice_size, folder, train_imgs, train_lbls, train_x_delta, 
                                                      train_y_delta, curr1, adj1, tolerance_factor)
                            
                    if ("bottom" in curr.keys() and "bottom"in adj.keys()):
                        for i in range(0, 121, int(120/enrich_factor)):
                            if i == 0:
                                continue
                            curr1 = pad_below(curr, curr["bottom"],i)
                            adj1 = pad_below(adj, adj["bottom"],i)
                            conc, x_delta, y_delta, x_file, y_file = VAL_concatenate_cubes(curr1, adj1, slice_size)
                            output = folder+"1="+x_file+"_"+str(curr1["top_row"])+"_"+str(curr1["left_col"])+"---"+y_file+"_"+str(adj1["top_row"])+"_"+str(adj1["left_col"])
                            # print(">>> MATCH >>>"+output)
                            save_img(output, conc)
                            # print(">>> >>> >>> SAVED")
                            train_imgs.append(output)
                            train_x_delta.append(x_delta)
                            train_y_delta.append(y_delta)
                            # mark the image as matched
                            train_lbls.append([0,1])

                            # TOLERANCE
                            VAL_add_tolerance_matches(slice_size, folder, train_imgs, train_lbls, train_x_delta, 
                                                      train_y_delta, curr1, adj1, tolerance_factor)
 
                # adding a condition for marking as not-matched - we mark only the "key" cubes which are every 250px
                # and not overlap - hence we reduce the ratio in favour of not matched which is enormous
                elif int(curr["row"]) == curr["row"] and int(adj["row"]) == adj["row"]: 
                    # mark the image as not matched
                    output = folder+"0="+x_file+"_"+str(curr["top_row"])+"_"+str(curr["left_col"])+"---"+y_file+"_"+str(adj["top_row"])+"_"+str(adj["left_col"])
                    # print("<<< nonmatch <<<"+output)
                    save_img(output, conc)
                    # print("<<< <<< <<< SAVED")
                    train_imgs.append(output)
                    train_lbls.append([1,0]) # not matched
                
                # discard not matched which are 
                else:
                    discard_c += 1
                    
    print("*** MATCHED="+str(sum(x[1] == 1 for x in train_lbls)))
    print("*** NOT MATCHED="+str(sum(x[0] == 1 for x in train_lbls)))
    print("*** DISCARDED="+str(discard_c))
    warnings.filterwarnings("default")
    return train_imgs, train_lbls, train_x_delta, train_y_delta

# convert the data structure of cubes into a test set of 2 arrays of images and labels
# each image is a concatanation of 2 images from the original cubes set, covering all combinations of images
# effectively creating Nx(N-1) images
def VAL_build_test_set(cubes, slice_size, folder, curr):
    for root, dirs, files in os.walk(folder):
        for f in files:
            os.unlink(os.path.join(root, f))
    # import pdb; pdb.set_trace()
    warnings.filterwarnings("ignore")

    train_imgs = []
    train_x_delta = []
    train_y_delta = []
    train_x_file = []
    train_y_file = []
    # iterate over the others (effectively n^2)
    # import pdb; pdb.set_trace()
    for adj in cubes:
        if (adj["file"] != curr["file"]) and curr["index"] < adj["index"]: # no need to test against self and avoid checking from both directions    
            # append the adjacent image to the current image
            conc, x_delta, y_delta, x_file, y_file = VAL_concatenate_cubes(curr, adj, slice_size)
            output = folder+x_file[:x_file.rfind(' ')]+"_"+str(curr["top_row"])+"_"+str(curr["left_col"])+"---"+y_file[:y_file.rfind(' ')]+"_"+str(adj["top_row"])+"_"+str(adj["left_col"])
            #import pdb; pdb.set_trace()
            np.save(output, conc)
            train_imgs.append(output)
            train_x_delta.append(x_delta)
            train_y_delta.append(y_delta)
            train_x_file.append(x_file)
            train_y_file.append(y_file)

    warnings.filterwarnings("default")
    return train_imgs,train_x_delta, train_y_delta, train_x_file, train_y_file

def frame_to_n_by_m(orig, start_vector, end_vector, is_col):
    max_val = np.amax(end_vector)
    min_val = np.amin(start_vector)
    width = max_val - min_val
    if (is_col):
        result = np.zeros((start_vector.size, width))
    else:
        result = np.zeros((width, start_vector.size))
    
    for i in range(0, start_vector.size):
        if (is_col):
            row_vec = orig[i, start_vector[i]:end_vector[i]]
        else:
            row_vec = orig[start_vector[i]:end_vector[i],i]
        temp = np.lib.pad(row_vec, (start_vector[i]-min_val, max_val-end_vector[i]), 'constant', constant_values=(0.09, 0.09))
        if (is_col):
            if (result[i].size != width):
                import pdb; pdb.set_trace()
            result[i] = temp[0:width]
        else:
            result[:,i] = temp[0:width]
    return min_val, result

def rough_tear_line(orig, start_vector, cut_mean, is_col, chew_factor):
    end_vector = np.empty(start_vector.size).astype(int)
    if (is_col and np.absolute(cut_mean-orig.shape[1]) < 10):
        end_vector.fill(orig.shape[1])
    elif (not is_col and np.absolute(cut_mean-orig.shape[0]) < 10):
        end_vector.fill(orig.shape[0])
    else:
        deviation_vector = np.random.normal(0, chew_factor, start_vector.size).astype(int)
        end_vector[0] = cut_mean + deviation_vector[0]
        for i in range(1, end_vector.size):
            end_vector[i] = end_vector[i - 1] + deviation_vector[i]
    
    start_px, cut_piece = frame_to_n_by_m(orig, start_vector, end_vector, is_col)    
    return start_px, cut_piece, end_vector

def rough_tear_image(image, cols, rows):
    pieces = []
    col_width = int(image.shape[1] / cols)
    row_height = int(image.shape[0] / rows)
    # print(col_width, row_height)
    next_col_start_vec = np.zeros((image.shape[0],), dtype=int)
    for col_idx in range(0, cols):
    #         import pdb; pdb.set_trace()
        start_col_px, cut_column, next_col_start_vec =  rough_tear_line(image, next_col_start_vec, col_width * (col_idx + 1), True, 5)
        next_row_start_vec = np.zeros((cut_column.shape[1],), dtype=int)
        for row_idx in range(0, rows):
            start_row_px, cut_piece, next_row_start_vec = rough_tear_line(cut_column, next_row_start_vec, row_height * (row_idx + 1), False, 1)

            ymin, ymax, xmin, xmax = calc_min_max_coordinates_dynamic(cut_piece, cutoff=BG_2_OBJ_RATIO)
            temp = crop(cut_piece, ymin, ymax, xmin, xmax)
            
            #import pdb; pdb.set_trace()
            piece = {}
            piece["orig"] = cut_piece
            piece["cut"] = temp
            piece["col"] = col_idx
            piece["row"] = row_idx
            piece["col_px"] = start_col_px + xmin
            piece["row_px"] = start_row_px + ymin
            pieces.append(piece)
            
    return pieces

In [5]:
# RUN Define model util functions

# initialize a shaped matrix of weights with random values
def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial)

# initialize a shaped matrix of bias with random values
def bias_variable(shape):
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial)

def conv2d(x, W):
  return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
  return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')

def max_pool_1x2(x):
  return tf.nn.max_pool(x, ksize=[1, 1, 2, 1],
                        strides=[1, 1, 2, 1], padding='SAME')

def max_pool_2x1(x):
  return tf.nn.max_pool(x, ksize=[1, 2, 1, 1],
                        strides=[1, 2, 1, 1], padding='SAME')

def max_pool_1x1(x):
  return tf.nn.max_pool(x, ksize=[1, 1, 1, 1],
                        strides=[1, 1, 1, 1], padding='SAME')

def max_pool_5x5(x):
  return tf.nn.max_pool(x, ksize=[1, 5, 5, 1],
                        strides=[1, 5, 5, 1], padding='SAME')

def max_pool_5x2(x):
  return tf.nn.max_pool(x, ksize=[1, 5, 2, 1],
                        strides=[1, 5, 2, 1], padding='SAME')


In [6]:
# RUN Image utility functions (external source)
def branchedPoints(skel):
    branch1=np.array([[2, 1, 2], [1, 1, 1], [2, 2, 2]])
    branch2=np.array([[1, 2, 1], [2, 1, 2], [1, 2, 1]])
    branch3=np.array([[1, 2, 1], [2, 1, 2], [1, 2, 2]])
    branch4=np.array([[2, 1, 2], [1, 1, 2], [2, 1, 2]])
    branch5=np.array([[1, 2, 2], [2, 1, 2], [1, 2, 1]])
    branch6=np.array([[2, 2, 2], [1, 1, 1], [2, 1, 2]])
    branch7=np.array([[2, 2, 1], [2, 1, 2], [1, 2, 1]])
    branch8=np.array([[2, 1, 2], [2, 1, 1], [2, 1, 2]])
    branch9=np.array([[1, 2, 1], [2, 1, 2], [2, 2, 1]])
    br1=mh.morph.hitmiss(skel,branch1)
    br2=mh.morph.hitmiss(skel,branch2)
    br3=mh.morph.hitmiss(skel,branch3)
    br4=mh.morph.hitmiss(skel,branch4)
    br5=mh.morph.hitmiss(skel,branch5)
    br6=mh.morph.hitmiss(skel,branch6)
    br7=mh.morph.hitmiss(skel,branch7)
    br8=mh.morph.hitmiss(skel,branch8)
    br9=mh.morph.hitmiss(skel,branch9)
    return br1+br2+br3+br4+br5+br6+br7+br8+br9

def endPoints(skel):
    endpoint1=np.array([[0, 0, 0],
                        [0, 1, 0],
                        [2, 1, 2]])
    
    endpoint2=np.array([[0, 0, 0],
                        [0, 1, 2],
                        [0, 2, 1]])
    
    endpoint3=np.array([[0, 0, 2],
                        [0, 1, 1],
                        [0, 0, 2]])
    
    endpoint4=np.array([[0, 2, 1],
                        [0, 1, 2],
                        [0, 0, 0]])
    
    endpoint5=np.array([[2, 1, 2],
                        [0, 1, 0],
                        [0, 0, 0]])
    
    endpoint6=np.array([[1, 2, 0],
                        [2, 1, 0],
                        [0, 0, 0]])
    
    endpoint7=np.array([[2, 0, 0],
                        [1, 1, 0],
                        [2, 0, 0]])
    
    endpoint8=np.array([[0, 0, 0],
                        [2, 1, 0],
                        [1, 2, 0]])
    
    ep1=mh.morph.hitmiss(skel,endpoint1)
    ep2=mh.morph.hitmiss(skel,endpoint2)
    ep3=mh.morph.hitmiss(skel,endpoint3)
    ep4=mh.morph.hitmiss(skel,endpoint4)
    ep5=mh.morph.hitmiss(skel,endpoint5)
    ep6=mh.morph.hitmiss(skel,endpoint6)
    ep7=mh.morph.hitmiss(skel,endpoint7)
    ep8=mh.morph.hitmiss(skel,endpoint8)
    ep = ep1+ep2+ep3+ep4+ep5+ep6+ep7+ep8
    return ep

def pruning(skeleton, size):
    '''remove iteratively end points "size" 
       times from the skeleton
    '''
    for i in range(0, size):
        endpoints = endPoints(skeleton)
        endpoints = np.logical_not(endpoints)
        skeleton = np.logical_and(skeleton,endpoints)
    return skeleton

def plot_comparison(original, filtered, filter_name):

    fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(8, 4), sharex=True, sharey=True)
    ax1.imshow(original, cmap=plt.cm.gray)
    ax1.set_title('original')
    ax1.axis('off')
    ax1.set_adjustable('box-forced')
    ax2.imshow(filtered, cmap=plt.cm.gray)
    ax2.set_title(filter_name)
    ax2.axis('off')
    ax2.set_adjustable('box-forced')
    


In [7]:
# RUN model_tf_deep - Define the model - 250, 125, 62, 25
def model_tf_deep(input_width, forced_bias=0): 
    global accuracy, correct_prediction, train_step, x, y_, y_conv, keep_prob, probability, probabilities #, W_fc, b_fc, cost, y_conv_temp
    
    # foundation of the model - the input layer of the image 250 x input_width*2
    x = tf.placeholder(tf.float32, [None, 250, input_width*2], "001")
    x_image = tf.reshape(x, [-1,250,input_width*2,1], "0011") # 1 is the number of color channels

    # the target digits of the model
    y_ = tf.placeholder(tf.float32, [None, 2], "002") # 1

    # zero convolutional layer: one input image and 32 output filters of 5x5
    W_conv0 = weight_variable([5, 5, 1, 32])
    b_conv0 = bias_variable([32])
    h_conv0 = tf.nn.relu(conv2d(x_image, W_conv0) + b_conv0, "0020")
    h_pool0 = max_pool_1x1(h_conv0) # size is maintained

    # first convolutional layer: one input image and 32 output filters of 5x5
    W_conv1 = weight_variable([5, 5, 32, 32])
#     W_conv1 = weight_variable([5, 5, 1, 32])
    b_conv1 = bias_variable([32])
    h_conv1 = tf.nn.relu(conv2d(h_pool0, W_conv1) + b_conv1, "0021")
#     h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1, "0021")
    if (input_width == 250):
        h_pool1 = max_pool_2x2(h_conv1) # size is reduced to 125x250
    elif (input_width == 125):
        h_pool1 = max_pool_2x1(h_conv1) # size is reduced to 125x250
    elif (input_width == 62):
        h_pool1 = max_pool_2x1(h_conv1) # size is reduced to 125x125
    elif (input_width == 25):
        h_pool1 = max_pool_2x1(h_conv1) # size is reduced to 125x50
    else:
        print("ERROR - unsupported slice width")
        return

    # second convolutional layer: 32 input (filtered) images and 32 output filters of 5x5
    W_conv2 = weight_variable([5, 5, 32, 32])
    b_conv2 = bias_variable([32])
    h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2, "0022")
    if (input_width == 62):
        h_pool2 = max_pool_1x1(h_conv2) # size is reduced to 125x125
    elif (input_width == 25):
        h_pool2 = max_pool_1x1(h_conv2) # size is reduced to 125x50
    else:
        h_pool2 = max_pool_1x2(h_conv2) # size is reduced to 125x125


    # third convolutional layer: 32 input (filtered) images and 32 output filters of 5x5
    W_conv3 = weight_variable([5, 5, 32, 32])
    b_conv3 = bias_variable([32])
    h_conv3 = tf.nn.relu(conv2d(h_pool2, W_conv3) + b_conv3, "0023")
    if (input_width == 25):
        h_pool3 = max_pool_5x2(h_conv3) # size is reduced to 25x25
    else:
        h_pool3 = max_pool_5x5(h_conv3) # size is reduced to 25x25


    h_pool3_flat = tf.reshape(h_pool3, [-1, 25*25*32]) # shape as an array 

    # fourth layer - fully connected with input 25*25*128 and output 1024
    W_fc1 = weight_variable([25*25*32, 1024])
    b_fc1 = bias_variable([1024])
    h_fc1 = tf.nn.relu(tf.matmul(h_pool3_flat, W_fc1) + b_fc1, "0024")

    # a drop layer with probability 
    keep_prob = tf.placeholder(tf.float32, name="003")
    h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob, name="0031")

    #     # final layer - reduce to one "class" for the linear regression
    #     W_fc = weight_variable([1024, 1]) 
    #     b_fc = bias_variable([1])         
    #     y_conv_temp = tf.matmul(h_fc1_drop, W_fc, name="0032") + b_fc
    #     y_conv = tf.minimum(y_conv_temp, tf.constant(BREAK_VAL, tf.float32))

    #     #     # minimize loss function
    #     #     cross_entropy = tf.reduce_mean(
    #     #       tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_conv))
    #     # cost = tf.reduce_sum(tf.pow(y_conv - y_, 2))/(2*BATCHES*BATCH_SIZE) # Mean squared error
    #     cost = tf.reduce_mean(tf.square(y_conv_temp - y_), name="0033") # Mean squared error

    #     #     # define train step and rate
    #     #     train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
    #     train_step = tf.train.AdamOptimizer(LEARNING_RATE).minimize(cost) # Gradient descent

    #     # evaluate the prediction and the accuracy on the train test - needed only for printing during the training
    #     correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
    #     accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    
    # final layer - softmax reduction 2 outputs
    W_fc2 = weight_variable([1024, 2])
    b_fc2 = bias_variable([2])
    c_fc2 = tf.constant([0, forced_bias], dtype=tf.float32)
    y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2 + c_fc2

    # minimize loss function
    cross_entropy = tf.reduce_mean(
      tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_conv))

    probability = tf.nn.softmax(y_conv,1)
    
    probabilities=tf.reduce_sum(probability,1)
    
    # define train step and rate
    train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

    # evaluate the prediction and the accuracy on the train test - needed only for printing during the training
    correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [8]:
# RUN model_tf_orig - Define the model - 250, 125, 62, 25
def model_tf_orig(input_width): 
    global accuracy, correct_prediction, train_step, x, y_, y_conv, keep_prob #, W_fc, b_fc, cost, y_conv_temp
    
    # foundation of the model - the input layer of the image 250 x input_width*2
    x = tf.placeholder(tf.float32, [None, 250, input_width*2], "001")
    x_image = tf.reshape(x, [-1,250,input_width*2,1], "0011") # 1 is the number of color channels

    # the target digits of the model
    y_ = tf.placeholder(tf.float32, [None, 2], "002") # 1

    # zero convolutional layer: one input image and 32 output filters of 5x5
#     W_conv0 = weight_variable([5, 5, 1, 32])
#     b_conv0 = bias_variable([32])
#     h_conv0 = tf.nn.relu(conv2d(x_image, W_conv0) + b_conv0, "0020")
#     h_pool0 = max_pool_1x1(h_conv0) # size is maintained

    # first convolutional layer: one input image and 32 output filters of 5x5
#     W_conv1 = weight_variable([5, 5, 32, 32])
    W_conv1 = weight_variable([5, 5, 1, 32])
    b_conv1 = bias_variable([32])
#     h_conv1 = tf.nn.relu(conv2d(h_pool0, W_conv1) + b_conv1, "0021")
    h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1, "0021")
    if (input_width == 250):
        h_pool1 = max_pool_2x2(h_conv1) # size is reduced to 125x250
    elif (input_width == 125):
        h_pool1 = max_pool_2x1(h_conv1) # size is reduced to 125x250
    elif (input_width == 62):
        h_pool1 = max_pool_2x1(h_conv1) # size is reduced to 125x125
    elif (input_width == 25):
        h_pool1 = max_pool_2x1(h_conv1) # size is reduced to 125x50
    else:
        print("ERROR - unsupported slice width")
        return

    # second convolutional layer: 32 input (filtered) images and 32 output filters of 5x5
    W_conv2 = weight_variable([5, 5, 32, 32])
    b_conv2 = bias_variable([32])
    h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2, "0022")
    if (input_width == 62):
        h_pool2 = max_pool_1x1(h_conv2) # size is reduced to 125x125
    elif (input_width == 25):
        h_pool2 = max_pool_1x1(h_conv2) # size is reduced to 125x50
    else:
        h_pool2 = max_pool_1x2(h_conv2) # size is reduced to 125x125


    # third convolutional layer: 32 input (filtered) images and 32 output filters of 5x5
    W_conv3 = weight_variable([5, 5, 32, 32])
    b_conv3 = bias_variable([32])
    h_conv3 = tf.nn.relu(conv2d(h_pool2, W_conv3) + b_conv3, "0023")
    if (input_width == 25):
        h_pool3 = max_pool_5x2(h_conv3) # size is reduced to 25x25
    else:
        h_pool3 = max_pool_5x5(h_conv3) # size is reduced to 25x25


    h_pool3_flat = tf.reshape(h_pool3, [-1, 25*25*32]) # shape as an array 

    # fourth layer - fully connected with input 25*25*128 and output 1024
    W_fc1 = weight_variable([25*25*32, 1024])
    b_fc1 = bias_variable([1024])
    h_fc1 = tf.nn.relu(tf.matmul(h_pool3_flat, W_fc1) + b_fc1, "0024")

    # a drop layer with probability 
    keep_prob = tf.placeholder(tf.float32, name="003")
    h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob, name="0031")

    #     # final layer - reduce to one "class" for the linear regression
    #     W_fc = weight_variable([1024, 1]) 
    #     b_fc = bias_variable([1])         
    #     y_conv_temp = tf.matmul(h_fc1_drop, W_fc, name="0032") + b_fc
    #     y_conv = tf.minimum(y_conv_temp, tf.constant(BREAK_VAL, tf.float32))

    #     #     # minimize loss function
    #     #     cross_entropy = tf.reduce_mean(
    #     #       tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_conv))
    #     # cost = tf.reduce_sum(tf.pow(y_conv - y_, 2))/(2*BATCHES*BATCH_SIZE) # Mean squared error
    #     cost = tf.reduce_mean(tf.square(y_conv_temp - y_), name="0033") # Mean squared error

    #     #     # define train step and rate
    #     #     train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
    #     train_step = tf.train.AdamOptimizer(LEARNING_RATE).minimize(cost) # Gradient descent

    #     # evaluate the prediction and the accuracy on the train test - needed only for printing during the training
    #     correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
    #     accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    
    # final layer - softmax reduction 2 outputs
    W_fc2 = weight_variable([1024, 2])
    b_fc2 = bias_variable([2])
    y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2

    # minimize loss function
    cross_entropy = tf.reduce_mean(
      tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_conv))

    # define train step and rate
    train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

    # evaluate the prediction and the accuracy on the train test - needed only for printing during the training
    correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [9]:
# RUN model_tf_wide - Define the model - 250, 125, 62, 25
def model_tf_wide(input_width): 
    global accuracy, correct_prediction, train_step, x, y_, y_conv, keep_prob #, W_fc, b_fc, cost, y_conv_temp
    
    # foundation of the model - the input layer of the image 250 x input_width*2
    x = tf.placeholder(tf.float32, [None, 250, input_width*2], "001")
    x_image = tf.reshape(x, [-1,250,input_width*2,1], "0011") # 1 is the number of color channels

    # the target digits of the model
    y_ = tf.placeholder(tf.float32, [None, 2], "002") # 1

    # first convolutional layer: one input image and 32 output filters of 5x5
    W_conv1 = weight_variable([5, 5, 1, 64])
    b_conv1 = bias_variable([64])
    h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1, "0021")
    if (input_width == 250):
        h_pool1 = max_pool_2x2(h_conv1) # size is reduced to 125x250
    elif (input_width == 125):
        h_pool1 = max_pool_2x1(h_conv1) # size is reduced to 125x250
    elif (input_width == 62):
        h_pool1 = max_pool_2x1(h_conv1) # size is reduced to 125x125
    elif (input_width == 25):
        h_pool1 = max_pool_2x1(h_conv1) # size is reduced to 125x50
    else:
        print("ERROR - unsupported slice width")
        return

    # second convolutional layer: 32 input (filtered) images and 32 output filters of 5x5
    W_conv2 = weight_variable([5, 5, 64, 64])
    b_conv2 = bias_variable([64])
    h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2, "0022")
    if (input_width == 62):
        h_pool2 = max_pool_1x1(h_conv2) # size is reduced to 125x125
    elif (input_width == 25):
        h_pool2 = max_pool_1x1(h_conv2) # size is reduced to 125x50
    else:
        h_pool2 = max_pool_1x2(h_conv2) # size is reduced to 125x125


    # third convolutional layer: 32 input (filtered) images and 32 output filters of 5x5
    W_conv3 = weight_variable([5, 5, 64, 64])
    b_conv3 = bias_variable([64])
    h_conv3 = tf.nn.relu(conv2d(h_pool2, W_conv3) + b_conv3, "0023")
    if (input_width == 25):
        h_pool3 = max_pool_5x2(h_conv3) # size is reduced to 25x25
    else:
        h_pool3 = max_pool_5x5(h_conv3) # size is reduced to 25x25


    h_pool3_flat = tf.reshape(h_pool3, [-1, 25*25*64]) # shape as an array 

    # fourth layer - fully connected with input 25*25*128 and output 1024
    W_fc1 = weight_variable([25*25*64, 2048])
    b_fc1 = bias_variable([2048])
    h_fc1 = tf.nn.relu(tf.matmul(h_pool3_flat, W_fc1) + b_fc1, "0024")

    # a drop layer with probability 
    keep_prob = tf.placeholder(tf.float32, name="003")
    h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob, name="0031")

    #     # final layer - reduce to one "class" for the linear regression
    #     W_fc = weight_variable([1024, 1]) 
    #     b_fc = bias_variable([1])         
    #     y_conv_temp = tf.matmul(h_fc1_drop, W_fc, name="0032") + b_fc
    #     y_conv = tf.minimum(y_conv_temp, tf.constant(BREAK_VAL, tf.float32))

    #     #     # minimize loss function
    #     #     cross_entropy = tf.reduce_mean(
    #     #       tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_conv))
    #     # cost = tf.reduce_sum(tf.pow(y_conv - y_, 2))/(2*BATCHES*BATCH_SIZE) # Mean squared error
    #     cost = tf.reduce_mean(tf.square(y_conv_temp - y_), name="0033") # Mean squared error

    #     #     # define train step and rate
    #     #     train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
    #     train_step = tf.train.AdamOptimizer(LEARNING_RATE).minimize(cost) # Gradient descent

    #     # evaluate the prediction and the accuracy on the train test - needed only for printing during the training
    #     correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
    #     accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    
    # final layer - softmax reduction 2 outputs
    W_fc2 = weight_variable([2048, 2])
    b_fc2 = bias_variable([2])
    y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2

    # minimize loss function
    cross_entropy = tf.reduce_mean(
      tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_conv))

    # define train step and rate
    train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

    # evaluate the prediction and the accuracy on the train test - needed only for printing during the training
    correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [10]:
# RUN train
def train(train_imgs, train_lbls, output_model, input_model=""):
    print("#####################################################################")
    print("TRAINING:")
    print("MODEL:"+output_model)
    print("#####################################################################")

    from random import randrange
    
    # TRAIN Prepare the session

    # create a saver object
    saver = tf.train.Saver()

    # start session and initialize variables
    sess = tf.InteractiveSession()
    if input_model != "":
        # Restore variables from disk.
        saver.restore(sess, input_model)
        print("Model restored.")
    else:
        sess.run(tf.initialize_all_variables())

    # TRAIN Train the model
    x_batch = []
    y_batch = []
    # run the train batches
    for i in range(BATCHES):
        x_batch = []
        y_batch = []
        for _ in range(BATCH_SIZE):
            random_index = randrange(0,len(train_imgs))
            image = np.load(train_imgs[random_index]+".npy")
            x_batch.append(image)
            y_batch.append(train_lbls[random_index])

        # train
        # print("step %d"%(i))
        train_step.run(feed_dict={x: x_batch, y_: y_batch, keep_prob: 0.5})

        # print the accuracy thus far
        if (i+1)%50 == 0:
            train_accuracy = accuracy.eval(feed_dict={
                x:x_batch, y_: y_batch, keep_prob: 1.0})
            print("step %d, training accuracy %g"%(i, train_accuracy))


    print("Optimization Finished!")
    train_accuracy = accuracy.eval(feed_dict={
        x:x_batch, y_: y_batch, keep_prob: 1.0})
    print("step %d, training accuracy %g"%(i, train_accuracy))

    # Save the variables to disk.
    save_path = saver.save(sess, output_model)
    print("Model saved in file: %s" % save_path)

    # Close the Session when we're done. If un-commented - need to run next bock of restore...
    sess.close()   
    print("#####################################################################")
    print("TRAINING ENDED")
    print("#####################################################################")
    print(" ")
    print(" ")    

In [11]:
# RUN pre_process test
def pre_process(folder, crops_data):
    print("#####################################################################")
    print("PRE_PROCESS:"+folder)
    print("#####################################################################")

    # READ crops
    crops = {}
    with open(crops_data) as csvfile:
        reader = csv.DictReader(csvfile, fieldnames=("file","x_start","y_start","x_end","y_end","rotate","horiz","upside"))
        next(reader)
        for row in reader:
            row['name'] = row['file'][row['file'].rfind('/')+1:]
            row['short_name'] = row['file'].split("/")[5] # stay with the folder name
            row['short_name'] = row['short_name'].split("-")
            row['short_name'] = row['short_name'][0] + row['short_name'][1] # set the short name...
            row['x_start'] = int(row['x_start'])
            row['col_px'] = int(row['x_start'])
            row['y_start'] = int(row['y_start'])
            row['row_px'] = int(row['y_start'])
            row['x_end'] = int(row['x_end'])
            row['y_end'] = int(row['y_end'])
            row['rotate'] = True if row['rotate'] == 'True' else False
            # import pdb; pdb.set_trace()
            crops[row['name']] = row

    result = []
    
    if files_in_gcp:
        folder = all_root + folder
        for file_ in folder_walker(folder):
            if 'P593' not in file_: # temp restriction
                # Read the image
                # image = img.imread(os.path.join(root, file_))
                # import pdb; pdb.set_trace()
                img_name = file_[file_.rfind('/')+1:file_.rfind('.')]
                image = gcp_np_load(file_)
                plt.imsave(os.path.join('temp/', img_name)+".png", image, cmap=plt.cm.gray) # needed for stage 18 - the alignment
                cubes = VAL_slice_to_static_slices(file_[file_.rfind('/')+1:], image, crops[img_name])
                print("File: %s >>> cubes: %d"%(file_, len(cubes)))
                result.extend(cubes)
        
    else:        
        folder = VOL_FOLDER + folder
        for root, dirs, files in os.walk(folder):
            for file_ in files:
                if file_[0:4] != 'P593': # temp restriction
                    # Read the image
                    # image = img.imread(os.path.join(root, file_))
                    img_name = file_[:file_.rfind('.')]
                    image = np.load(os.path.join(root, file_))
                    plt.imsave(os.path.join(folder, file_)+".png", image, cmap=plt.cm.gray)
                    # import pdb; pdb.set_trace()
                    cubes = VAL_slice_to_static_slices(file_, image, crops[img_name])
                    print("File: %s >>> cubes: %d"%(file_, len(cubes)))
                    result.extend(cubes)
    
    return result
    print("#####################################################################")
    print("PRE_PROCESS ENDED")
    print("#####################################################################")
    print(" ")
    print(" ")

In [12]:
# RUN pre_process_training - crop image, then tear it randomly to various tears, then per tear create cubes out of the edges, return cube set
def pre_process_training(img_name, x_start=X_START, x_end=X_END, y_start=Y_START, y_end=Y_END):
    print("#####################################################################")
    print("PRE_PROCESS:"+img_name)
    print("#####################################################################")
    short_name = img_name[:img_name.rfind('-D')]
    image = read_and_crop(img_name, x_start, x_end, y_start, y_end)
    result = []
    
    # clean the output fragments folder before writing a new set of artificial fragments into it
    for root, dirs, files in os.walk(ROOT_FOLDER+"fragments/"):
        for f in files:
            os.unlink(os.path.join(root, f))

    
    for col_cut in range(3, 9): # 3...10
        for row_cut in range(2, 6): # 2...5
            print("PRE_PROCESS:::"+"TEAR_"+str(col_cut)+"X"+str(row_cut))
            pieces = rough_tear_image(image, col_cut, row_cut)
            
            for piece in pieces:
                # print("PRE_PROCESS:::"+"PIECE_"+str(piece["col"])+"X"+str(piece["row"]))
                fragment_name = short_name + "_TEAR_"+str(col_cut)+"X"+str(row_cut)+"_PIECE_"+str(piece["col"])+"X"+str(piece["row"])
                fragment_file_name = short_name + "_"+str(col_cut)+"X"+str(row_cut)+"_"+str(piece["col"])+"X"+str(piece["row"])
                # import pdb; pdb.set_trace()
                plt.imsave(os.path.join(ROOT_FOLDER+"fragments/",fragment_file_name+".jpg"), piece["cut"], cmap=plt.cm.gray)
                cubes = VAL_slice_TEAR_to_static_slices(fragment_name, piece)
                for cube in cubes:
                    cube["tear"] = str(col_cut)+"X"+str(row_cut)
                    cube["piece_col"] = piece["col"]
                    cube["piece_row"] = piece["row"]
                # print("File: %s >>> cubes: %d"%(file_, len(cubes)))
                result.extend(cubes)
    
    return result
    print("#####################################################################")
    print("PRE_PROCESS ENDED")
    print("#####################################################################")
    print(" ")
    print(" ")

In [13]:
def validate1(cubes, model, slice_size, folder, curr_cube):    
    # VALIDATE prepare the data sets
    test_imgs, test_x_delta, test_y_delta, test_x_file, test_y_file = VAL_build_test_set(cubes, slice_size, folder, curr_cube)
    print("loaded %d images"%(len(test_imgs)))

In [14]:
def helper_name_extractor(first, second):
    second = second.split(".")[0] # get rid of the suffix

    # split the first file name and calc the row and col
    split = first.split("_")
    firstNameOrig = split[0]
    firstX = int(split[2])
    firstCol = 0 if firstX < 100 else 1
    split = firstNameOrig.split("-")
    first = split[0] + split[1]

    # split the second file name and calc the row and col
    split = second.split("_")
    secondNameOrig = split[0]
    secondX = int(split[2])
    secondCol = 0 if secondX < 100 else 1
    split = secondNameOrig.split("-")
    second = split[0] + split[1]

    fragmentKey = first + "_" + second

    fragmentAndSideKey = first + "_" + str(firstCol) \
        + "_" + second + "_" + str(secondCol)
        
    return fragmentKey, fragmentAndSideKey
    
def helper_split_name(concatName):
    # /home/il239838/files/train_concats3/PX303-Fg006-V-C02-R02_TEAR_5X3_PIECE_3X2_870_1745---PX303-Fg006-V-C02-R02_TEAR_5X3_PIECE_4X2_620_50
    concatName = concatName[concatName.rfind('/')+1:]

    # PX303-Fg006-V-C02-R02_TEAR_5X3_PIECE_3X2_870_1745---PX303-Fg006-V-C02-R02_TEAR_5X3_PIECE_4X2_620_50
    firstName = concatName[0:concatName.find('---')]
    secondName = concatName[concatName.find('---')+3:]

    return firstName, secondName

In [15]:
def validate2(folder, model, slice_size):
    test_imgs = []
    test_x_file = []
    test_y_file = []
    the_root = ""
    for root, dirs, files in os.walk(folder):
        the_root = root
        for file_ in files:
            test_imgs.append( os.path.join(root, file_) )
            test_x_file.append(file_[:file_.rfind('---P')])
            test_y_file.append(file_[file_.rfind('---P')+3:])
            
    print(len(test_imgs))
    
    # VALIDATE Prepare a test session 

    # Add ops to save and restore all the variables.
    saver = tf.train.Saver()

    # start session and initialize variables
    sess = tf.InteractiveSession()

    # Restore variables from disk.
    saver.restore(sess, model)
    print("Model restored.")

    # VALIDATE Validate the model

    # import pdb; pdb.set_trace()
    v1t = []
    count = 0
    length = len(test_imgs)
    batch = 50
    x_batch = []
    # change the ranges in the loop below - first number is the start point (multiplied by batch size)
    # second number is the end point (multiplied by batch size)
    # third number is the jump from batch to batch
    # use the length about to set the batch length
    for start in range(0, length, batch):
        for i in range(start, start+batch):
            if (i < length):
                image = np.load(test_imgs[i])
                x_batch.append(image)
                count += 1

        # import pdb; pdb.set_trace()
        # print("Validating start at #%d end at %d"%(start*batch,(start+length)*batch))
        my_prediction=tf.argmax(y_conv,1)
        v1 = my_prediction.eval(feed_dict={x:x_batch, keep_prob: 1.0})
        v1t = np.concatenate((v1t, v1), axis=0)
        x_batch = []
        if (start+batch) % 500 == 0:
            print(">>> step %d"%(start+batch))


    match_indexes = np.nonzero(v1t)[0]
    A = np.array(test_x_file)
    B = np.array(test_y_file)
    C = np.array(test_imgs)
    match_x_files = A[match_indexes]
    match_y_files = B[match_indexes]
    match_images = C[match_indexes]
    
    # CURRENTLY no need in saving the matched cubes since we can settle for the stats of the matched cubes    
    #     for matched_img in match_images:
    #         load_img = np.load(matched_img)
    #         plt.imsave(os.path.join(VOL_FOLDER+ "matched/",matched_img[matched_img.rfind('/')+1:]+".png"), load_img, cmap=plt.cm.gray)
    
    for root, dirs, files in os.walk(folder):
        for file_ in files:
            os.remove( os.path.join(root, file_) ) # delete it from the FS
         
    print("FOUND Matches #: " + str(len(match_indexes)))    
    
    #     with open('pairs_all.csv', 'a') as csvfile:
    #         csvout = csv.writer(csvfile)
    #         for idx, test_img in enumerate(test_imgs):
    #             # print("MATCH %s === %s"%(test_imgs[match_index], train_lbls[match_index]))
    #             match_class = 0
    #             if idx in match_indexes:
    #                 match_class = 1
    #             csvout.writerow([test_img, match_class])
    #             # plt.imsave("match_"+match_index+".jpg", C[match_index])

    for idx, test_img in enumerate(test_imgs):
        first_name, second_name = helper_split_name(test_img)
        fragmentKey, framentAndSideKey = helper_name_extractor(first_name, second_name)
        if fragmentKey not in fragment_counters:
            fragment_counters[fragmentKey] = 0
        if framentAndSideKey not in fragment_and_side_counters:
            fragment_and_side_counters[framentAndSideKey] = 0
        fragment_counters[fragmentKey] += 1
        fragment_and_side_counters[framentAndSideKey] += 1
        
    with open('pairs_matches.csv', 'a') as csvfile:
        csvout = csv.writer(csvfile)
        for match_index in match_indexes:
            fragmentKey, framentAndSideKey = helper_name_extractor(test_x_file[match_index], test_y_file[match_index])
            csvout.writerow([test_x_file[match_index], test_y_file[match_index], fragment_counters[fragmentKey], fragment_and_side_counters[framentAndSideKey]])

    # Close the Session when we're done.
    sess.close()   

In [16]:
def validate2_for_cross_validation(test_imgs, test_lbls, model, max_samples=0):
    print(len(test_imgs))
    
    # VALIDATE Prepare a test session 

    # Add ops to save and restore all the variables.
    saver = tf.train.Saver()

    # start session and initialize variables
    sess = tf.InteractiveSession()

    # Restore variables from disk.
    saver.restore(sess, model)
    print("Model restored.")

    # VALIDATE Validate the model

    count = 0
    se = 0
    st = 0
    v1t = []
    v2t = []
    v1tt = []
    v2tt = []
    length = len(test_imgs)
    if max_samples != 0:
        length = max_samples
    batch = 100
    x_batch = []
    y_batch = []
    # change the ranges in the loop below - first number is the start point (multiplied by batch size)
    # second number is the end point (multiplied by batch size)
    # third number is the jump from batch to batch
    # use the length about to set the batch length
    for start in range(0, length, batch):
        for i in range(start, start+batch):
            if (i < length):
                image = np.load(test_imgs[i]+".npy")
                x_batch.append(image)
                y_batch.append(train_lbls[i])                
                
        # print the accuracy thus far
    #         train_accuracy = accuracy.eval(feed_dict={
    #             x:x_batch, y_: y_batch, keep_prob: 1.0})
    #         print("step %d, training accuracy %g"%(i, train_accuracy))

        # print("Validating start at #%d end at %d"%(start*batch,(start+length)*batch))
    #         my_prediction=tf.argmax(y_conv,1)
    #         v1 = my_prediction.eval(feed_dict={x:x_batch, keep_prob: 1.0})
    #         v1t = np.concatenate((v1t, v1), axis=0)

        ######## printing the predictions and their normalized values
        # print("y_conv="+str(y_conv.eval(feed_dict={x:x_batch, y_: y_batch, keep_prob: 1.0})))
        # print("probability="+str(probability.eval(feed_dict={x:x_batch, y_: y_batch, keep_prob: 1.0})))
        # print("probabilities="+str(probabilities.eval(feed_dict={x:x_batch, y_: y_batch, keep_prob: 1.0})))
        
        my_prediction=tf.argmax(y_conv,1)
        my_target=tf.argmax(y_,1)
        v1 = my_prediction.eval(feed_dict={x:x_batch, y_: y_batch, keep_prob: 1.0})
        v2 = my_target.eval(feed_dict={x:x_batch, y_: y_batch, keep_prob: 1.0})
        v1t = np.concatenate((v1t, v1), axis=0)
        v2t = np.concatenate((v2t, v2), axis=0)

        c1 = np.sum(np.absolute(np.subtract(v2, v1)))
        c2 = np.sum(np.absolute(v2))
        se += c1
        st += c2

        x_batch = []
        y_batch = []
        print(">>> step %d"%(start+batch))
        
        count += ((i+1) - start)
        precision, recall, f_score, support = precision_recall_fscore_support(v2t, v1t, average='binary')
        print("step %d-%d, precision %f, recall %f, f_score %f"%(start, i, precision, recall, f_score))
        # print("Accumulated total true = %d"%(st));
        # print("Accumulated total error rate = %f"%(se/count));
        # v1tt = np.concatenate((v1tt, v1t), axis=0)
        # v2tt = np.concatenate((v2tt, v2t), axis=0)
        print("=== total %d match %d"%(count, len(np.nonzero(v1t)[0])))

    precision, recall, f_score, support = precision_recall_fscore_support(v2t, v1t, average='binary')
    print("TOTAL %d, precision %f, recall %f, f_score %f"%(count, precision, recall, f_score))
    print("TOTAL true = %d"%(st));
    print("TOTAL error rate = %f"%(se/count));
    
    match_indexes = np.nonzero(v1t)[0]
    C = np.array(test_imgs)
    match_images = C[match_indexes]
    
    # for matched_img in match_images:
    #    load_img = np.load(matched_img+".npy")
    #    plt.imsave(os.path.join(ROOT_FOLDER+"synt_matched/",matched_img[matched_img.rfind('/')+1:]+".png"), load_img, cmap=plt.cm.gray)

    with open('synt_all.csv', 'a') as csvfile:
        csvout = csv.writer(csvfile)
        for idx, test_img in enumerate(test_imgs):
            # print("MATCH %s === %s"%(test_imgs[match_index], train_lbls[match_index]))
            match_class = 0
            if idx in match_indexes:
                match_class = 1
            csvout.writerow([test_img, train_lbls[idx], match_class, is_enriched[idx]])
            # plt.imsave("match_"+match_index+".jpg", C[match_index])

    with open('synt_matches.csv', 'a') as csvfile:
        csvout = csv.writer(csvfile)
        for match_index in match_indexes:
            # print("MATCH %s === %s"%(test_imgs[match_index], train_lbls[match_index]))
            csvout.writerow([test_imgs[match_index], train_lbls[match_index], is_enriched[match_index]])
            # plt.imsave("match_"+match_index+".jpg", C[match_index])

    # Close the Session when we're done.
    sess.close()   

In [17]:
def iter_validate(cubes, model, slice_size, folder):
    cubes_len = len(cubes)
    print("#####################################################################")
    print("VALIDATING ", cubes_len, " cubes")
    print("#####################################################################")
    
    count = 0
    # iterate over the cubes
    for curr in cubes:
        count += 1
        if count > 0:  # should be zero #199 and count < 300: ### TEMP LIMITATION
            print("### %s ### CUBE:%s"%(count, curr["file"]+"_"+str(curr["top_row"])+"_"+str(curr["left_col"])))
            validate1(cubes, model, slice_size, folder, curr)
            validate2(folder, model, slice_size)
            if count%10 == 0:
                tf.reset_default_graph()
                model_tf_deep(250, 0)  # set 1 instead of 0 in order to allow more grace - roughly reduce from 50% to 40% threashold  
                print("############ R E S E T #############")
    print("#####################################################################")
    print("VALIDATION ENDED")
    print("#####################################################################")
    print(" ")
    print(" ")

In [18]:
def run_all(folder, model, slice_size):
    model_tf(slice_size)
    cubes_set = pre_process(folder)
    validate(cubes_set, model, slice_size)

In [ ]:
# HELPER block
# image = read_and_crop("PX303/FG001/PX303-Fg001-V-C01-R01-D05032015-T112602-ML924__012.jpg")
## image = read_and_crop("PX303/FG004/PX303-Fg004-V-C01-R01-D08032015-T110900-ML924__012.jpg", 100, -1, 400, -1)
# image = read_and_crop("PX303/FG004/PX303-Fg004-V-C01-R02-D08032015-T105147-ML924__012.jpg")
# image = read_and_crop("PX303/FG004/PX303-Fg004-V-C02-R01-D08032015-T110025-ML924__012.jpg")
# image = read_and_crop("PX303/FG004/PX303-Fg004-V-C02-R02-D08032015-T105553-ML924__012.jpg")
# image = read_and_crop("PX303/FG006/PX303-Fg006-V-C01-R01-D08032015-T120605-ML924__012.jpg")
# image = read_and_crop("PX303/FG006/PX303-Fg006-V-C01-R02-D08032015-T115230-ML924__012.jpg")
# image = read_and_crop("PX303/FG006/PX303-Fg006-V-C02-R01-D08032015-T120158-ML924__012.jpg")
##image = read_and_crop("PX303/FG006/PX303-Fg006-V-C02-R02-D08032015-T115704-ML924__012.jpg", 0, 6200, 0, 4400)
##plt.imshow(image)

In [ ]:
def load_train_from_disk(path):
    train_imgs = []
    train_lbls = []
    for root, dirs, files in os.walk(path):
        for file_ in files:
            file_name = os.path.join(root, file_)
            file_name = file_name[:file_name.rfind(".")]
            train_imgs.append(file_name)
            train_lbls.append([1,0] if file_.startswith("0=") else [0,1])
    return train_imgs, train_lbls

In [ ]:
# TEST - take test-set and cross-validate on it (uncomment all for full test run)
# cubes_set = pre_process_training("PX303-Fg006-V-C02-R02-D08032015-T115622-ML638__006.jpg", 0, 6200, 0, 4400)
#import pdb; pdb.set_trace()
cubes_set = pre_process("cropped/", "crops.csv") # cropped==front
# cubes_set = pre_process("cropped2_np/", "crops.csv")

#####################################################################
PRE_PROCESS:cropped/
#####################################################################
Blob thesis-papyri/cropped/P589-Fg001-R-C01-R01-D12022013-T110710-ML638 _018.jpg.npy downloaded to temp/P589-Fg001-R-C01-R01-D12022013-T110710-ML638 _018.jpg.npy.
File: thesis-papyri/cropped/P589-Fg001-R-C01-R01-D12022013-T110710-ML638 _018.jpg.npy >>> cubes: 39
Blob thesis-papyri/cropped/P589-Fg002-R-C01-R01-D12022013-T111734-ML638 _018.jpg.npy downloaded to temp/P589-Fg002-R-C01-R01-D12022013-T111734-ML638 _018.jpg.npy.
File: thesis-papyri/cropped/P589-Fg002-R-C01-R01-D12022013-T111734-ML638 _018.jpg.npy >>> cubes: 98
Blob thesis-papyri/cropped/P589-Fg003-R-C01-R01-D12022013-T113754-ML638 _018.jpg.npy downloaded to temp/P589-Fg003-R-C01-R01-D12022013-T113754-ML638 _018.jpg.npy.
File: thesis-papyri/cropped/P589-Fg003-R-C01-R01-D12022013-T113754-ML638 _018.jpg.npy >>> cubes: 86
Blob thesis-papyri/cropped/P589-Fg004-R-C01-R01-D1

File: thesis-papyri/cropped/P590-Fg019-R-C01-R01-D14022013-T114714-ML638 _018.jpg.npy >>> cubes: 34
Blob thesis-papyri/cropped/P590-Fg020-R-C01-R01-D14022013-T115732-ML638 _018.jpg.npy downloaded to temp/P590-Fg020-R-C01-R01-D14022013-T115732-ML638 _018.jpg.npy.
File: thesis-papyri/cropped/P590-Fg020-R-C01-R01-D14022013-T115732-ML638 _018.jpg.npy >>> cubes: 8
Blob thesis-papyri/cropped/P590-Fg021-R-C01-R01-D14022013-T120752-ML638 _018.jpg.npy downloaded to temp/P590-Fg021-R-C01-R01-D14022013-T120752-ML638 _018.jpg.npy.
File: thesis-papyri/cropped/P590-Fg021-R-C01-R01-D14022013-T120752-ML638 _018.jpg.npy >>> cubes: 6
Blob thesis-papyri/cropped/P590-Fg022-R-C01-R01-D14022013-T121616-ML638 _018.jpg.npy downloaded to temp/P590-Fg022-R-C01-R01-D14022013-T121616-ML638 _018.jpg.npy.
File: thesis-papyri/cropped/P590-Fg022-R-C01-R01-D14022013-T121616-ML638 _018.jpg.npy >>> cubes: 6
Blob thesis-papyri/cropped/P590-Fg023-R-C01-R01-D14022013-T122534-ML638 _018.jpg.npy downloaded to temp/P590-Fg023

/home/ilan/venv35/lib/python3.5/site-packages/numpy/core/fromnumeric.py:3118: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/ilan/venv35/lib/python3.5/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


File: thesis-papyri/cropped/P596-Fg004-R-C01-R01-D19012014-T122330-ML638 _018.jpg.npy >>> cubes: 40
Blob thesis-papyri/cropped/P596-Fg005-R-C01-R01-D19012014-T130628-ML638 _018.jpg.npy downloaded to temp/P596-Fg005-R-C01-R01-D19012014-T130628-ML638 _018.jpg.npy.
File: thesis-papyri/cropped/P596-Fg005-R-C01-R01-D19012014-T130628-ML638 _018.jpg.npy >>> cubes: 28
Blob thesis-papyri/cropped/P596-Fg006-R-C01-R01-D19012014-T131539-ML638 _018.jpg.npy downloaded to temp/P596-Fg006-R-C01-R01-D19012014-T131539-ML638 _018.jpg.npy.
File: thesis-papyri/cropped/P596-Fg006-R-C01-R01-D19012014-T131539-ML638 _018.jpg.npy >>> cubes: 32
Blob thesis-papyri/cropped/P596-Fg007-R-C01-R01-D19012014-T132505-ML638 _018.jpg.npy downloaded to temp/P596-Fg007-R-C01-R01-D19012014-T132505-ML638 _018.jpg.npy.
File: thesis-papyri/cropped/P596-Fg007-R-C01-R01-D19012014-T132505-ML638 _018.jpg.npy >>> cubes: 36
Blob thesis-papyri/cropped/P596-Fg008-R-C01-R01-D19012014-T133443-ML638 _018.jpg.npy downloaded to temp/P596-Fg

Blob thesis-papyri/cropped/P596-Fg036-R-C01-R01-D20012014-T120247-ML638 _018.jpg.npy downloaded to temp/P596-Fg036-R-C01-R01-D20012014-T120247-ML638 _018.jpg.npy.
File: thesis-papyri/cropped/P596-Fg036-R-C01-R01-D20012014-T120247-ML638 _018.jpg.npy >>> cubes: 30
Blob thesis-papyri/cropped/P596-Fg037-R-C01-R01-D20012014-T121105-ML638 _018.jpg.npy downloaded to temp/P596-Fg037-R-C01-R01-D20012014-T121105-ML638 _018.jpg.npy.
File: thesis-papyri/cropped/P596-Fg037-R-C01-R01-D20012014-T121105-ML638 _018.jpg.npy >>> cubes: 30
Blob thesis-papyri/cropped/P596-Fg038-R-C01-R01-D20012014-T121918-ML638 _018.jpg.npy downloaded to temp/P596-Fg038-R-C01-R01-D20012014-T121918-ML638 _018.jpg.npy.
File: thesis-papyri/cropped/P596-Fg038-R-C01-R01-D20012014-T121918-ML638 _018.jpg.npy >>> cubes: 28
Blob thesis-papyri/cropped/P596-Fg039-R-C01-R01-D20012014-T131543-ML638 _018.jpg.npy downloaded to temp/P596-Fg039-R-C01-R01-D20012014-T131543-ML638 _018.jpg.npy.
File: thesis-papyri/cropped/P596-Fg039-R-C01-R01

File: thesis-papyri/cropped/P596-Fg067-R-C01-R01-D22012014-T115503-ML638 _018.jpg.npy >>> cubes: 28
Blob thesis-papyri/cropped/P596-Fg068-R-C01-R01-D22012014-T121348-ML638 _018.jpg.npy downloaded to temp/P596-Fg068-R-C01-R01-D22012014-T121348-ML638 _018.jpg.npy.
File: thesis-papyri/cropped/P596-Fg068-R-C01-R01-D22012014-T121348-ML638 _018.jpg.npy >>> cubes: 19
Blob thesis-papyri/cropped/P596-Fg069-R-C01-R01-D22012014-T122323-ML638 _018.jpg.npy downloaded to temp/P596-Fg069-R-C01-R01-D22012014-T122323-ML638 _018.jpg.npy.
File: thesis-papyri/cropped/P596-Fg069-R-C01-R01-D22012014-T122323-ML638 _018.jpg.npy >>> cubes: 30
Blob thesis-papyri/cropped/P596-Fg070-R-C01-R01-D22012014-T123218-ML638 _018.jpg.npy downloaded to temp/P596-Fg070-R-C01-R01-D22012014-T123218-ML638 _018.jpg.npy.
File: thesis-papyri/cropped/P596-Fg070-R-C01-R01-D22012014-T123218-ML638 _018.jpg.npy >>> cubes: 26
Blob thesis-papyri/cropped/P596-Fg071-R-C01-R01-D22012014-T124029-ML638 _018.jpg.npy downloaded to temp/P596-Fg

Blob thesis-papyri/cropped/P597-Fg014-R-C01-R01-D30112014-T131452-ML638 _018.jpg.npy downloaded to temp/P597-Fg014-R-C01-R01-D30112014-T131452-ML638 _018.jpg.npy.
File: thesis-papyri/cropped/P597-Fg014-R-C01-R01-D30112014-T131452-ML638 _018.jpg.npy >>> cubes: 20
Blob thesis-papyri/cropped/P597-Fg015-R-C01-R01-D30112014-T132316-ML638 _018.jpg.npy downloaded to temp/P597-Fg015-R-C01-R01-D30112014-T132316-ML638 _018.jpg.npy.
File: thesis-papyri/cropped/P597-Fg015-R-C01-R01-D30112014-T132316-ML638 _018.jpg.npy >>> cubes: 26
Blob thesis-papyri/cropped/P597-Fg016-R-C01-R01-D30112014-T133128-ML638 _018.jpg.npy downloaded to temp/P597-Fg016-R-C01-R01-D30112014-T133128-ML638 _018.jpg.npy.
File: thesis-papyri/cropped/P597-Fg016-R-C01-R01-D30112014-T133128-ML638 _018.jpg.npy >>> cubes: 42
Blob thesis-papyri/cropped/P597-Fg017-R-C01-R01-D30112014-T133944-ML638 _018.jpg.npy downloaded to temp/P597-Fg017-R-C01-R01-D30112014-T133944-ML638 _018.jpg.npy.
File: thesis-papyri/cropped/P597-Fg017-R-C01-R01

Blob thesis-papyri/cropped/P597-Fg046-R-C01-R01-D01122014-T112623-ML638 _018.jpg.npy downloaded to temp/P597-Fg046-R-C01-R01-D01122014-T112623-ML638 _018.jpg.npy.
File: thesis-papyri/cropped/P597-Fg046-R-C01-R01-D01122014-T112623-ML638 _018.jpg.npy >>> cubes: 15
Blob thesis-papyri/cropped/P597-Fg047-R-C01-R01-D01122014-T113433-ML638 _018.jpg.npy downloaded to temp/P597-Fg047-R-C01-R01-D01122014-T113433-ML638 _018.jpg.npy.
File: thesis-papyri/cropped/P597-Fg047-R-C01-R01-D01122014-T113433-ML638 _018.jpg.npy >>> cubes: 32
Blob thesis-papyri/cropped/P597-Fg048-R-C01-R01-D01122014-T122119-ML638 _018.jpg.npy downloaded to temp/P597-Fg048-R-C01-R01-D01122014-T122119-ML638 _018.jpg.npy.
File: thesis-papyri/cropped/P597-Fg048-R-C01-R01-D01122014-T122119-ML638 _018.jpg.npy >>> cubes: 24
Blob thesis-papyri/cropped/P597-Fg049-R-C01-R01-D01122014-T123402-ML638 _018.jpg.npy downloaded to temp/P597-Fg049-R-C01-R01-D01122014-T123402-ML638 _018.jpg.npy.
File: thesis-papyri/cropped/P597-Fg049-R-C01-R01

File: thesis-papyri/cropped/P598-Fg005-R-C01-R01-D07012014-T113140-ML638 _018.jpg.npy >>> cubes: 56
Blob thesis-papyri/cropped/P598-Fg006-R-C01-R01-D07012014-T113917-ML638 _018.jpg.npy downloaded to temp/P598-Fg006-R-C01-R01-D07012014-T113917-ML638 _018.jpg.npy.
File: thesis-papyri/cropped/P598-Fg006-R-C01-R01-D07012014-T113917-ML638 _018.jpg.npy >>> cubes: 17
Blob thesis-papyri/cropped/P598-Fg007-R-C01-R01-D07012014-T114651-ML638 _018.jpg.npy downloaded to temp/P598-Fg007-R-C01-R01-D07012014-T114651-ML638 _018.jpg.npy.
File: thesis-papyri/cropped/P598-Fg007-R-C01-R01-D07012014-T114651-ML638 _018.jpg.npy >>> cubes: 14
Blob thesis-papyri/cropped/P598-Fg008-R-C01-R01-D07012014-T123530-ML638 _018.jpg.npy downloaded to temp/P598-Fg008-R-C01-R01-D07012014-T123530-ML638 _018.jpg.npy.
File: thesis-papyri/cropped/P598-Fg008-R-C01-R01-D07012014-T123530-ML638 _018.jpg.npy >>> cubes: 36
Blob thesis-papyri/cropped/P598-Fg009-R-C01-R01-D07012014-T124317-ML638 _018.jpg.npy downloaded to temp/P598-Fg

In [ ]:
tf.reset_default_graph()
model_tf_deep(250, 0)  # set 1 instead of 0 in order to allow more grace - roughly reduce from 50% to 40% threashold  

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [ ]:
#os.remove('pairs_matches.csv')
fragment_counters = {}
fragment_and_side_counters = {}

In [ ]:
iter_validate(cubes_set, ROOT_FOLDER + "models/front_tear_model3.ckpt", CUBE_SIZE, VOL_FOLDER+ "mixed/")

#####################################################################
VALIDATING  6919  cubes
#####################################################################
### 1 ### CUBE:P589-Fg001-R-C01-R01-D12022013-T110710-ML638 _018.jpg.npy_550_782
loaded 6880 images
6865
INFO:tensorflow:Restoring parameters from /media/1KGB_ILAN/papyrus/models/front_tear_model3.ckpt
Model restored.
>>> step 500
>>> step 1000
>>> step 1500
>>> step 2000
>>> step 2500
>>> step 3000
>>> step 3500
>>> step 4000
>>> step 4500
>>> step 5000
>>> step 5500
>>> step 6000
>>> step 6500
FOUND Matches #: 340
### 2 ### CUBE:P589-Fg001-R-C01-R01-D12022013-T110710-ML638 _018.jpg.npy_300_50
loaded 6880 images
6865
INFO:tensorflow:Restoring parameters from /media/1KGB_ILAN/papyrus/models/front_tear_model3.ckpt
Model restored.
>>> step 500
>>> step 1000
>>> step 1500
>>> step 2000
>>> step 2500
>>> step 3000
>>> step 3500
>>> step 4000
>>> step 4500
>>> step 5000
>>> step 5500
>>> step 6000
>>> step 6500
FOUND Matches #: 6

loaded 6880 images
6865
INFO:tensorflow:Restoring parameters from /media/1KGB_ILAN/papyrus/models/front_tear_model3.ckpt
Model restored.
>>> step 500
>>> step 1000
>>> step 1500
>>> step 2000
>>> step 2500
>>> step 3000
>>> step 3500
>>> step 4000
>>> step 4500
>>> step 5000
>>> step 5500
>>> step 6000
>>> step 6500
FOUND Matches #: 801
############ R E S E T #############
### 21 ### CUBE:P589-Fg001-R-C01-R01-D12022013-T110710-ML638 _018.jpg.npy_250_50
loaded 6880 images
6865
INFO:tensorflow:Restoring parameters from /media/1KGB_ILAN/papyrus/models/front_tear_model3.ckpt
Model restored.
>>> step 500
>>> step 1000
>>> step 1500
>>> step 2000
>>> step 2500
>>> step 3000
>>> step 3500
>>> step 4000
>>> step 4500
>>> step 5000
>>> step 5500
>>> step 6000
>>> step 6500
FOUND Matches #: 330
### 22 ### CUBE:P589-Fg001-R-C01-R01-D12022013-T110710-ML638 _018.jpg.npy_850_50
loaded 6880 images
6865
INFO:tensorflow:Restoring parameters from /media/1KGB_ILAN/papyrus/models/front_tear_model3.ckpt
Mo

>>> step 500
>>> step 1000
>>> step 1500
>>> step 2000
>>> step 2500
>>> step 3000
>>> step 3500
>>> step 4000
>>> step 4500
>>> step 5000
>>> step 5500
>>> step 6000
>>> step 6500
FOUND Matches #: 633
### 40 ### CUBE:P589-Fg002-R-C01-R01-D12022013-T111734-ML638 _018.jpg.npy_2000_1452
loaded 6782 images
6767
INFO:tensorflow:Restoring parameters from /media/1KGB_ILAN/papyrus/models/front_tear_model3.ckpt
Model restored.
>>> step 500
>>> step 1000
>>> step 1500
>>> step 2000
>>> step 2500
>>> step 3000
>>> step 3500
>>> step 4000
>>> step 4500
>>> step 5000
>>> step 5500
>>> step 6000
>>> step 6500
FOUND Matches #: 122
############ R E S E T #############
### 41 ### CUBE:P589-Fg002-R-C01-R01-D12022013-T111734-ML638 _018.jpg.npy_950_1452
loaded 6782 images
6767
INFO:tensorflow:Restoring parameters from /media/1KGB_ILAN/papyrus/models/front_tear_model3.ckpt
Model restored.
>>> step 500
>>> step 1000
>>> step 1500
>>> step 2000
>>> step 2500
>>> step 3000
>>> step 3500
>>> step 4000
>>> ste

>>> step 4500
>>> step 5000
>>> step 5500
>>> step 6000
>>> step 6500
FOUND Matches #: 1302
### 59 ### CUBE:P589-Fg002-R-C01-R01-D12022013-T111734-ML638 _018.jpg.npy_1100_50
loaded 6782 images
6767
INFO:tensorflow:Restoring parameters from /media/1KGB_ILAN/papyrus/models/front_tear_model3.ckpt
Model restored.
>>> step 500
>>> step 1000
>>> step 1500
>>> step 2000
>>> step 2500
>>> step 3000
>>> step 3500
>>> step 4000
>>> step 4500
>>> step 5000
>>> step 5500
>>> step 6000
>>> step 6500
FOUND Matches #: 109
### 60 ### CUBE:P589-Fg002-R-C01-R01-D12022013-T111734-ML638 _018.jpg.npy_0_50
loaded 6782 images
6767
INFO:tensorflow:Restoring parameters from /media/1KGB_ILAN/papyrus/models/front_tear_model3.ckpt
Model restored.
>>> step 500
>>> step 1000
>>> step 1500
>>> step 2000
>>> step 2500
>>> step 3000
>>> step 3500
>>> step 4000
>>> step 4500
>>> step 5000
>>> step 5500
>>> step 6000
>>> step 6500
FOUND Matches #: 571
############ R E S E T #############
### 61 ### CUBE:P589-Fg002-R-C01

loaded 6782 images
6767
INFO:tensorflow:Restoring parameters from /media/1KGB_ILAN/papyrus/models/front_tear_model3.ckpt
Model restored.
>>> step 500
>>> step 1000
>>> step 1500
>>> step 2000
>>> step 2500
>>> step 3000
>>> step 3500
>>> step 4000
>>> step 4500
>>> step 5000
>>> step 5500
>>> step 6000
>>> step 6500
FOUND Matches #: 1274
### 79 ### CUBE:P589-Fg002-R-C01-R01-D12022013-T111734-ML638 _018.jpg.npy_1550_1452
loaded 6782 images
6767
INFO:tensorflow:Restoring parameters from /media/1KGB_ILAN/papyrus/models/front_tear_model3.ckpt
Model restored.
>>> step 500
>>> step 1000
>>> step 1500
>>> step 2000
>>> step 2500
>>> step 3000
>>> step 3500
>>> step 4000
>>> step 4500
>>> step 5000
>>> step 5500
>>> step 6000
>>> step 6500
FOUND Matches #: 197
### 80 ### CUBE:P589-Fg002-R-C01-R01-D12022013-T111734-ML638 _018.jpg.npy_400_1452
loaded 6782 images
6767
INFO:tensorflow:Restoring parameters from /media/1KGB_ILAN/papyrus/models/front_tear_model3.ckpt
Model restored.
>>> step 500
>>> 

...

loaded 3015 images
3015
INFO:tensorflow:Restoring parameters from /media/1KGB_ILAN/papyrus/models/front_tear_model3.ckpt
Model restored.
>>> step 500
>>> step 1000
>>> step 1500
>>> step 2000
>>> step 2500
>>> step 3000
FOUND Matches #: 3
### 3886 ### CUBE:P596-Fg074-R-C01-R01-D22012014-T131052-ML638 _018.jpg.npy_100_31
loaded 3015 images
3015
INFO:tensorflow:Restoring parameters from /media/1KGB_ILAN/papyrus/models/front_tear_model3.ckpt
Model restored.
>>> step 500
>>> step 1000
>>> step 1500
>>> step 2000
>>> step 2500
>>> step 3000
FOUND Matches #: 8
### 3887 ### CUBE:P596-Fg074-R-C01-R01-D22012014-T131052-ML638 _018.jpg.npy_100_50
loaded 3015 images
3015
INFO:tensorflow:Restoring parameters from /media/1KGB_ILAN/papyrus/models/front_tear_model3.ckpt
Model restored.
>>> step 500
>>> step 1000
>>> step 1500
>>> step 2000
>>> step 2500
>>> step 3000
FOUND Matches #: 17
### 3888 ### CUBE:P596-Fg074-R-C01-R01-D22012014-T131052-ML638 _018.jpg.npy_0_31
loaded 3015 images
3015
INFO:tensor

Model restored.
>>> step 500
>>> step 1000
>>> step 1500
>>> step 2000
>>> step 2500
>>> step 3000
FOUND Matches #: 773
############ R E S E T #############
### 3911 ### CUBE:P596-Fg075-R-C01-R01-D22012014-T131929-ML638 _018.jpg.npy_400_264
loaded 2997 images
2997
INFO:tensorflow:Restoring parameters from /media/1KGB_ILAN/papyrus/models/front_tear_model3.ckpt
Model restored.
>>> step 500
>>> step 1000
>>> step 1500
>>> step 2000
>>> step 2500
>>> step 3000
FOUND Matches #: 505
### 3912 ### CUBE:P596-Fg075-R-C01-R01-D22012014-T131929-ML638 _018.jpg.npy_350_264
loaded 2997 images
2997
INFO:tensorflow:Restoring parameters from /media/1KGB_ILAN/papyrus/models/front_tear_model3.ckpt
Model restored.
>>> step 500
>>> step 1000
>>> step 1500
>>> step 2000
>>> step 2500
>>> step 3000
FOUND Matches #: 60
### 3913 ### CUBE:P596-Fg075-R-C01-R01-D22012014-T131929-ML638 _018.jpg.npy_150_50
loaded 2997 images
2997
INFO:tensorflow:Restoring parameters from /media/1KGB_ILAN/papyrus/models/front_tear_mo

Model restored.
>>> step 500
>>> step 1000
>>> step 1500
>>> step 2000
>>> step 2500
>>> step 3000
FOUND Matches #: 14
### 3936 ### CUBE:P596-Fg077-R-C01-R01-D22012014-T133640-ML638 _018.jpg.npy_250_50
loaded 2973 images
2973
INFO:tensorflow:Restoring parameters from /media/1KGB_ILAN/papyrus/models/front_tear_model3.ckpt
Model restored.
>>> step 500
>>> step 1000
>>> step 1500
>>> step 2000
>>> step 2500
>>> step 3000
FOUND Matches #: 648
### 3937 ### CUBE:P596-Fg077-R-C01-R01-D22012014-T133640-ML638 _018.jpg.npy_550_50
loaded 2973 images
2973
INFO:tensorflow:Restoring parameters from /media/1KGB_ILAN/papyrus/models/front_tear_model3.ckpt
Model restored.
>>> step 500
>>> step 1000
>>> step 1500
>>> step 2000
>>> step 2500
>>> step 3000
FOUND Matches #: 1
### 3938 ### CUBE:P596-Fg077-R-C01-R01-D22012014-T133640-ML638 _018.jpg.npy_500_50
loaded 2973 images
2973
INFO:tensorflow:Restoring parameters from /media/1KGB_ILAN/papyrus/models/front_tear_model3.ckpt
Model restored.
>>> step 500
>>

>>> step 2500
FOUND Matches #: 411
############ R E S E T #############
### 3961 ### CUBE:P596-Fg079-R-C01-R01-D22012014-T135356-ML638 _018.jpg.npy_50_50
loaded 2937 images
2937
INFO:tensorflow:Restoring parameters from /media/1KGB_ILAN/papyrus/models/front_tear_model3.ckpt
Model restored.
>>> step 500
>>> step 1000
>>> step 1500
>>> step 2000
>>> step 2500
FOUND Matches #: 117
### 3962 ### CUBE:P596-Fg079-R-C01-R01-D22012014-T135356-ML638 _018.jpg.npy_400_50
loaded 2937 images
2937
INFO:tensorflow:Restoring parameters from /media/1KGB_ILAN/papyrus/models/front_tear_model3.ckpt
Model restored.
>>> step 500
>>> step 1000
>>> step 1500
>>> step 2000
>>> step 2500
FOUND Matches #: 132
### 3963 ### CUBE:P596-Fg079-R-C01-R01-D22012014-T135356-ML638 _018.jpg.npy_250_187
loaded 2937 images
2937
INFO:tensorflow:Restoring parameters from /media/1KGB_ILAN/papyrus/models/front_tear_model3.ckpt
Model restored.
>>> step 500
>>> step 1000
>>> step 1500
>>> step 2000
>>> step 2500
FOUND Matches #: 0


loaded 2915 images
2915
INFO:tensorflow:Restoring parameters from /media/1KGB_ILAN/papyrus/models/front_tear_model3.ckpt
Model restored.
>>> step 500
>>> step 1000
>>> step 1500
>>> step 2000
>>> step 2500
FOUND Matches #: 2
### 3988 ### CUBE:P596-Fg080-R-C01-R01-D22012014-T140215-ML638 _018.jpg.npy_100_19
loaded 2915 images
2915
INFO:tensorflow:Restoring parameters from /media/1KGB_ILAN/papyrus/models/front_tear_model3.ckpt
Model restored.
>>> step 500
>>> step 1000
>>> step 1500
>>> step 2000
>>> step 2500
FOUND Matches #: 0
### 3989 ### CUBE:P596-Fg080-R-C01-R01-D22012014-T140215-ML638 _018.jpg.npy_100_50
loaded 2915 images
2915
INFO:tensorflow:Restoring parameters from /media/1KGB_ILAN/papyrus/models/front_tear_model3.ckpt
Model restored.
>>> step 500
>>> step 1000
>>> step 1500
>>> step 2000
>>> step 2500
FOUND Matches #: 1
### 3990 ### CUBE:P596-Fg080-R-C01-R01-D22012014-T140215-ML638 _018.jpg.npy_0_19
loaded 2915 images
2915
INFO:tensorflow:Restoring parameters from /media/1KGB_

In [28]:
Out

{}

In [ ]:
#### STOP

In [ ]:
# RE-TEST - take test-set and cross-validate on it (uncomment all for full test run)
# train_imgs, train_lbls = \
#     load_train_from_disk(ROOT_FOLDER + "train_concats3/")
# tf.reset_default_graph()
# model_tf_deep(250, 1)    
# validate2_for_cross_validation(train_imgs, train_lbls, ROOT_FOLDER + "model_binary/tear_model2.ckpt")

In [ ]:
#len(cubes_set)

In [ ]:
Out

In [ ]:
for cube in cubes_set:
    if (cube['cube'].shape[0] < 250 or cube['cube'].shape[1] < 250):
        print(cube['file'])
        print(cube['cube'].shape)
#     import pdb; pdb.set_trace()